In [12]:
#후처리, 평가 추가했습니다(오류확인후 수정 예정)

# 1.1: 라이브러리 설치 (필요시 주석 해제)
!pip install torch torchvision
!pip install timm
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install geopandas rasterio

# 1.2: 기본 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import glob
import numpy as np
import math
import copy
import inspect
import functools
import logging
from typing import Callable, Dict, List, Optional, Tuple, Union

# 1.3: 모델 코드에 필요한 라이브러리 임포트
import fvcore.nn.weight_init as weight_init
from fvcore.common.config import CfgNode as _CfgNode
from timm.layers import trunc_normal_, to_2tuple, DropPath
from torch.cuda.amp import autocast
from torch.nn.init import xavier_uniform_, constant_, uniform_, normal_
import torch.utils.checkpoint as checkpoint
from torch import Tensor

# 1.4: (참고) 데이터 전처리에 사용했던 라이브러리
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.features import rasterize
from shapely.geometry import box

  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-ko2dd_98
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-ko2dd_98
  Resolved https://github.com/facebookresearch/fvcore to commit 70e69fb3ff7a82631f6a82061a7650c4f11da876
  Preparing metadata (setup.py) ... done


In [13]:
# 2. 데이터 전처리 (별도 실행 가정)
# ------------------------------------

def convert_to_target_format(label_mask):
    """
    [헬퍼 함수]
    '통합 라벨 마스크'([H, W])를 모델 학습에 필요한 'targets' 딕셔너리 형식으로 변환합니다.
    이 함수는 GIS 데이터(정수 ID)를 딥러닝 모델이 이해할 수 있는 형식(이진 마스크)으로 번역하는 역할을 합니다.
    """
    # 통합 마스크에서 고유한 건물 ID(1, 2, 3...)를 모두 찾아냅니다. 배경 값인 0은 제외합니다.
    instance_ids = torch.unique(label_mask)
    instance_ids = instance_ids[instance_ids != 0]

    # 만약 현재 타일에 건물이 하나도 없다면(ID가 없다면), 학습 시 오류가 나지 않도록
    # 내용이 비어있는 텐서를 가진 딕셔너리를 반환합니다.
    if len(instance_ids) == 0:
        return {'instance_class': torch.tensor([], dtype=torch.long), 'mask': torch.tensor([], dtype=torch.uint8)}

    #'브로드캐스팅'이라는 텐서 연산을 사용하여, 각 건물 ID에 해당하는 픽셀만 1(True)이고
    # 나머지는 0(False)인 개별 마스크들을 효율적으로 생성합니다.
    # 결과적으로 [N, H, W] 크기의 3차원 텐서가 만들어집니다. (N: 건물 개수)
    masks = (label_mask == instance_ids[:, None, None])

    # 현재 프로젝트에서는 '건물'이라는 단일 클래스만 사용하므로,
    # 건물 개수(N)만큼 0으로 채워진 클래스 라벨 텐서를 생성합니다.
    instance_classes = torch.zeros(len(instance_ids), dtype=torch.long)

    # 팀원이 요청한 최종 형식에 맞춰 'instance_class'와 'mask'를 키로 하는 딕셔너리를 구성하여 반환합니다.
    return {'instance_class': instance_classes, 'masks': masks.to(torch.uint8)}


def create_final_dataset_per_file(
    ortho_image_path,
    building_shp_path,
    output_dir,
    tile_size=1024
):
    """
    [RAM/스토리지 문제 해결 최종 버전]
    GIS 데이터를 가공하여, 이미지 타일과 그에 1:1로 대응하는 'targets' 딕셔너리를
    각각 개별 파일로 저장하여 RAM 문제를 해결하는 최종 전처리 함수임

    [중요] 왜 '딕셔너리의 리스트' ([{...}, {...}, ...])를 만들지 않는가?
    ----------------------------------------------------------------
    이 '딕셔너리의 리스트' 형식은 모든 라벨 딕셔너리를 하나의 거대한 리스트 변수에
    담아두었다가 마지막에 한 번에 파일로 저장하는 방식임

    발생했던 문제:
    1. 메모리 초과 (RAM OOM): 'targets' 딕셔너리는 내부에 [N, H, W] 크기의 큰 마스크 텐서를
       포함하여 용량이 매우 큽니다. 수천 개의 딕셔너리를 하나의 리스트에 모두 담으면
       Colab의 제한된 RAM(12GB)을 순식간에 초과하여 커널 재시작 오류가 발생했음
    2. 저장 비효율: 최종적으로 생성되는 단일 리스트 파일의 크기가 매우 커져
       Google Drive의 저장 공간도 빠르게 소진됨

    현재 코드의 해결책:
    이 함수는 한 번에 하나의 타일만 처리하고, 생성된 이미지와 라벨 딕셔너리를 즉시
    개별 파일로 저장함. 이 '파일 단위 저장' 방식은 메모리 사용량을 최소화하여
    Colab 환경에서도 대용량 데이터를 안정적으로 처리함
    ----------------------------------------------------------------
    """
    # 결과물이 저장될 'images'와 'labels_dict' 폴더를 생성합니다.
    img_tile_dir = os.path.join(output_dir, "images")
    lbl_dict_dir = os.path.join(output_dir, "labels_dict")
    os.makedirs(img_tile_dir, exist_ok=True)
    os.makedirs(lbl_dict_dir, exist_ok=True)

    # GeoPandas로 건물 shapefile을, Rasterio로 정사영상(.tif)을 엽니다.
    gdf_buildings = gpd.read_file(building_shp_path)

    with rasterio.open(ortho_image_path) as src:
        # 영상 전체를 1024x1024 타일로 깔끔하게 나누기 위해 가로/세로 크기를 1024의 배수로 조정합니다.
        width = (src.width // tile_size) * tile_size
        height = (src.height // tile_size) * tile_size
        print(f"전체 영상 크기: ({src.width}, {src.height}), 조정된 크기: ({width}, {height})")

        # 만약 두 데이터의 좌표계(CRS)가 다르다면, 건물 데이터를 정사영상 기준으로 통일합니다.
        if gdf_buildings.crs != src.crs:
            gdf_buildings = gdf_buildings.to_crs(src.crs)

        # 생성된 파일 개수를 세기 위한 카운터입니다.
        tile_count = 0
        # 이중 for문을 사용해 전체 영상을 1024x1024 크기의 타일(window) 단위로 순회합니다.
        for j in range(0, height, tile_size):
            for i in range(0, width, tile_size):
                window = Window(i, j, tile_size, tile_size)
                tile_transform = src.window_transform(window)

                # 현재 타일의 지리적 좌표 경계를 계산하고, 이 경계와 겹치는 건물만 빠르게 찾아냅니다.
                tile_bounds = rasterio.windows.bounds(window, src.transform)
                tile_bbox = box(*tile_bounds)
                intersecting_buildings = gdf_buildings[gdf_buildings.intersects(tile_bbox)]

                # 현재 타일 안에 건물이 하나라도 있을 경우에만 데이터 생성 및 저장을 진행합니다.
                if not intersecting_buildings.empty:
                    # 1. '통합 라벨 마스크'를 메모리 상에서 생성합니다. (배경=0, 건물=1, 2, 3...)
                    simple_ids = range(1, len(intersecting_buildings) + 1)
                    shapes = [(geom, id) for geom, id in zip(intersecting_buildings.geometry, simple_ids)]

                    label_mask = rasterize(
                        shapes=shapes,
                        out_shape=(tile_size, tile_size),
                        transform=tile_transform,
                        fill=0,
                        all_touched=True,
                        dtype=rasterio.int32
                    )

                    # 2. 위에서 생성된 '통합 라벨 마스크'를 즉시 'targets' 딕셔너리로 변환합니다.
                    target_dict = convert_to_target_format(torch.from_numpy(label_mask).long())

                    # 3. 최종 결과물인 이미지와 'targets' 딕셔너리를 1:1로 매칭되는 개별 파일로 즉시 저장합니다.
                    image_tile = src.read(window=window)
                    if image_tile.shape[0] > 3:
                        image_tile = image_tile[:3, :, :]\

                    image_tensor = torch.from_numpy(image_tile).float()

                    torch.save(image_tensor, os.path.join(img_tile_dir, f"tile_{tile_count}.pt"))
                    torch.save(target_dict, os.path.join(lbl_dict_dir, f"tile_{tile_count}.pt"))

                    # 파일 저장 후 카운터를 1 증가시킵니다.
                    tile_count += 1

        print(f"\n--- 최종 데이터셋 생성 완료 ---")
        print(f"총 {tile_count}개의 [이미지 파일]과 [라벨 딕셔너리 파일] 쌍을 생성했습니다.")
        print(f"이미지 저장 위치: {img_tile_dir}")
        print(f"라벨 딕셔너리 저장 위치: {lbl_dict_dir}")



# 3단계: 함수 실행


# --- Google Drive 경로 설정 ---
# QGIS에서 전처리한 원본 데이터 파일들의 경로입니다.
ORTHO_TIF_PATH = "/content/drive/MyDrive/gis_project/clipped_정사영상.tif"
BUILDING_SHP_PATH = "/content/drive/MyDrive/gis_project/clipped_buildings.shp"
# 최종 결과물이 저장될 폴더의 경로입니다.
OUTPUT_DIR = "/content/drive/MyDrive/gis_project/final_dataset_per_file"
# --------------------------------

# 위에서 정의한 최종 데이터셋 생성 함수를 실행합니다.
# (참고: 이 코드는 학습 노트북과는 별도로 *먼저* 실행되어야 합니다.)
# create_final_dataset_per_file(ORTHO_TIF_PATH, BUILDING_SHP_PATH, OUTPUT_DIR)

# preprocess_gis_data_ipynb.ipynb 파일은
# 이 학습 코드와 별도로 *미리 실행*되어야 합니다.
#
# 아래 코드는 Google Drive의 '/content/drive/MyDrive/gis_project/final_dataset_per_file' 경로에
# 이미지 타일(images/tile_X.pt)과 라벨 딕셔너리(labels_dict/tile_X.pt)가
# 이미 저장되어 있다고 가정하고 진행합니다.
# ------------------------------------

In [14]:
# 1.1: GIS 라이브러리 설치
!pip install rasterio geopandas

# 1.2: 더미데이터 생성 코드
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import box
from rasterio.transform import from_origin

print("--- 1. 테스트용 TIF/SHP 파일 생성을 시작합니다. ---")

# --- 설정값 ---
TIF_FILENAME = "dummy_image.tif"
SHP_FILENAME = "dummy_buildings.shp"
IMAGE_SIZE = 1500  # 1024x1024 타일 1개가 나오도록 1024보다 크게 설정
TARGET_CRS = "EPSG:3857"

# --- 1. 가짜 TIF 이미지 생성 (3-Band, 1500x1500) ---
np_image = np.random.randint(0, 256, (3, IMAGE_SIZE, IMAGE_SIZE), dtype=np.uint8)
# Y좌표 범위: [north - (height * ysize), north] -> [-500, 1000]
# X좌표 범위: [west, west + (width * xsize)] -> [1000, 2500]
geo_transform = from_origin(west=1000.0, north=1000.0, xsize=1.0, ysize=1.0)

with rasterio.open(
    TIF_FILENAME, 'w', driver='GTiff', height=IMAGE_SIZE, width=IMAGE_SIZE,
    count=3, dtype=np_image.dtype, crs=TARGET_CRS, transform=geo_transform,
) as dst:
    dst.write(np_image)
print(f"✅ [1/2] 가짜 이미지 '{TIF_FILENAME}' 생성 완료")

# --- 2. 가짜 SHP 파일 생성 (Polygon) ---
# TIF 영역 (X: 1000~2500, Y: -500~1000) 안에 건물 2개 생성
# Box 1: (X: 1100~1200, Y: 800~900)
building_1 = box(1100, 800, 1200, 900)
# Box 2: (X: 1500~1600, Y: 300~500)
building_2 = box(1500, 300, 1600, 500)

gdf = gpd.GeoDataFrame(
    {'id': [1, 2]}, geometry=[building_1, building_2], crs=TARGET_CRS
)
gdf.to_file(SHP_FILENAME, driver='ESRI Shapefile')

print(f"✅ [2/2] 가짜 건물 '{SHP_FILENAME}' 생성 완료")
print("\n--- 🏁 모든 테스트 파일 생성 완료 ---")

--- 1. 테스트용 TIF/SHP 파일 생성을 시작합니다. ---
✅ [1/2] 가짜 이미지 'dummy_image.tif' 생성 완료
✅ [2/2] 가짜 건물 'dummy_buildings.shp' 생성 완료

--- 🏁 모든 테스트 파일 생성 완료 ---


In [15]:
# (2번 셀의 앞부분 ... 함수 정의 ... 는 그대로 둡니다)
# ... def create_final_dataset_per_file(...) ...
# ... (생략) ...
# ------------------------------------

# 3단계: 함수 실행

# --- Google Drive 경로 설정 (대신, 방금 만든 더미 파일 사용) ---
ORTHO_TIF_PATH = "dummy_image.tif"
BUILDING_SHP_PATH = "dummy_buildings.shp"
# 최종 결과물이 저장될 폴더의 경로입니다.
OUTPUT_DIR = "dummy_dataset_output"
# --------------------------------

# 위에서 정의한 최종 데이터셋 생성 함수를 실행합니다.
# (수정) 주석을 해제하여 실행합니다.
print("--- 테스트 데이터로 전처리를 시작합니다... ---")
create_final_dataset_per_file(ORTHO_TIF_PATH, BUILDING_SHP_PATH, OUTPUT_DIR)
print("--- 테스트 데이터 전처리 완료. ---")

# (이후 ... (참고: 이 코드는...) 부분은 그대로 둡니다)
# ...

--- 테스트 데이터로 전처리를 시작합니다... ---
전체 영상 크기: (1500, 1500), 조정된 크기: (1024, 1024)

--- 최종 데이터셋 생성 완료 ---
총 1개의 [이미지 파일]과 [라벨 딕셔너리 파일] 쌍을 생성했습니다.
이미지 저장 위치: dummy_dataset_output/images
라벨 딕셔너리 저장 위치: dummy_dataset_output/labels_dict
--- 테스트 데이터 전처리 완료. ---


In [16]:
# --- 3. 🧠 모델 헬퍼 함수 및 클래스 ---

from typing import Optional

# @title ShapeSpec (pixeldecoder_tem_fpn.ipynb)
class ShapeSpec:
    """
    (backbone과 pixeldecoder에서 모두 사용)
    """
    def __init__(self, channels=None, height=None, width=None, stride=None):
        self.channels = channels
        self.height = height
        self.width = width
        self.stride = stride

    def __str__(self) -> str:
        return f"ShapeSpec(C={self.channels}, H={self.height}, W={self.width}, S={self.stride})"

    __repr__ = __str__

# @title _get_clones (pixeldecoder_tem_fpn.ipynb)
def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

# @title _get_activation_fn (pixeldecoder_tem_fpn.ipynb)
def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(f"activation should be relu/gelu, not {activation}.")

# @title get_norm (pixeldecoder_tem_fpn.ipynb)
def get_norm(norm, out_channels):
    if norm is None:
        return None
    if isinstance(norm, str):
        if norm == "" or norm.lower() == "none":
            return None
        if norm == "BN":
            return nn.BatchNorm2d(out_channels)
        if norm == "SyncBN":
            return nn.SyncBatchNorm(out_channels)
        if norm == "GN":
            groups = 32 if out_channels % 32 == 0 else max(1, min(32, out_channels))
            return nn.GroupNorm(groups, out_channels)
        if norm == "LN":
            class _ChannelsFirstLayerNorm(nn.Module):
                def __init__(self, num_channels, eps=1e-6):
                    super().__init__()
                    self.weight = nn.Parameter(torch.ones(num_channels))
                    self.bias = nn.Parameter(torch.zeros(num_channels))
                    self.eps = eps
                def forward(self, x):
                    mean = x.mean(dim=1, keepdim=True)
                    var = (x - mean).pow(2).mean(dim=1, keepdim=True)
                    x = (x - mean) / torch.sqrt(var + self.eps)
                    return x * self.weight[:, None, None] + self.bias[:, None, None]
            return _ChannelsFirstLayerNorm(out_channels)
        raise ValueError(...)
    if callable(norm):
        return norm(out_channels)
    if isinstance(norm, nn.Module):
        return norm
    raise TypeError(...)

# @title Conv2d (pixeldecoder_tem_fpn.ipynb)
class Conv2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
        norm=None,
        activation=None,
    ):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=bias if norm is None else False,
        )
        self.norm = norm
        self.activation = activation

    @property
    def weight(self):
        return self.conv.weight

    @property
    def bias(self):
        return self.conv.bias

    def forward(self, x):
        x = self.conv(x)
        if self.norm is not None:
            x = self.norm(x)
        if self.activation is not None:
            x = self.activation(x) if callable(self.activation) else self.activation(x)
        return x

# @title configurable (pixeldecoder_tem_fpn.ipynb)
# fvcore의 configurable 데코레이터를 위한 헬퍼 함수들
def configurable(init_func=None, *, from_config=None):
    if init_func is not None:
        assert (
            inspect.isfunction(init_func)
            and from_config is None
            and init_func.__name__ == "__init__"
        ), "Incorrect use of @configurable. Check API documentation for examples."

        @functools.wraps(init_func)
        def wrapped(self, *args, **kwargs):
            try:
                from_config_func = type(self).from_config
            except AttributeError as e:
                raise AttributeError(
                    "Class with @configurable must have a 'from_config' classmethod."
                ) from e
            if not inspect.ismethod(from_config_func):
                raise TypeError("Class with @configurable must have a 'from_config' classmethod.")

            if _called_with_cfg(*args, **kwargs):
                explicit_args = _get_args_from_config(from_config_func, *args, **kwargs)
                init_func(self, **explicit_args)
            else:
                init_func(self, *args, **kwargs)

        return wrapped
    else:
        if from_config is None:
            return configurable
        assert inspect.isfunction(
            from_config
        ), "from_config argument of configurable must be a function!"

        def wrapper(orig_func):
            @functools.wraps(orig_func)
            def wrapped(*args, **kwargs):
                if _called_with_cfg(*args, **kwargs):
                    explicit_args = _get_args_from_config(from_config, *args, **kwargs)
                    return orig_func(**explicit_args)
                else:
                    return orig_func(*args, **kwargs)

            wrapped.from_config = from_config
            return wrapped

        return wrapper

def _get_args_from_config(from_config_func, *args, **kwargs):
    signature = inspect.signature(from_config_func)
    if list(signature.parameters.keys())[0] != "cfg":
        if inspect.isfunction(from_config_func):
            name = from_config_func.__name__
        else:
            name = f"{from_config_func.__self__}.from_config"
        raise TypeError(f"{name} must take 'cfg' as the first argument!")
    support_var_arg = any(
        param.kind in [param.VAR_POSITIONAL, param.VAR_KEYWORD]
        for param in signature.parameters.values()
    )
    if support_var_arg:
        ret = from_config_func(*args, **kwargs)
    else:
        supported_arg_names = set(signature.parameters.keys())
        extra_kwargs = {}
        for name in list(kwargs.keys()):
            if name not in supported_arg_names:
                extra_kwargs[name] = kwargs.pop(name)
        ret = from_config_func(*args, **kwargs)
        ret.update(extra_kwargs)
    return ret


def _called_with_cfg(*args, **kwargs):
    # omegaconf는 설치가 필요할 수 있습니다.
    try:
        from omegaconf import DictConfig
    except ImportError:
        # 설치되지 않았을 경우를 대비한 임시 클래스
        class DictConfig:
            pass

    if len(args) and isinstance(args[0], (_CfgNode, DictConfig)):
        return True
    if isinstance(kwargs.pop("cfg", None), (_CfgNode, DictConfig)):
        return True
    return False

In [17]:
# 4. 모델 정의: Backbone (Swin Transformer)
# (backbone코드.ipynb 파일에서 복사)

# 이미지 쪼개기
def window_partition(x,window_size):
  B,H,W,C = x.shape
  x=x.view(B,H//window_size,window_size,W//window_size,window_size,C)
  windows=x.permute(0,1,3,2,4,5).contiguous().view(-1,window_size,window_size,C)
  return windows

# 이미지 합치기
def window_reverse(windows,window_size,H,W):
  B = int(windows.shape[0] / (H * W / window_size / window_size))
  x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
  x = x.permute(0,1,3,2,4,5).contiguous().view(B, H, W, -1)
  return x

# 이미지 -> 토큰으로 변형
class PatchEmbed(nn.Module):
    def __init__(self, patch_size=4, in_chans=3, embed_dim=96,norm_layer=None):
        super().__init__()
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size) #이미지 분할
        self.norm = norm_layer

    def forward(self, x):
        _, _, H, W = x.size()
        if W % self.patch_size != 0:
            x = F.pad(x, (0, self.patch_size - W % self.patch_size))
        if H % self.patch_size != 0:
            x = F.pad(x, (0, 0, 0, self.patch_size - H % self.patch_size))
        x = self.proj(x)
        return x

# 해상도 절반, 피처 2배
class PatchMerging(nn.Module):
  def __init__(self,dim,norm_layer=nn.LayerNorm):
    super().__init__()
    self.dim = dim
    self.reduction = nn.Linear(4*dim,2*dim,bias=False)
    self.norm = norm_layer(4*dim)

  def forward(self,x,H,W):
    B,L,C=x.shape
    assert L==H*W, "Swin PatchMerging클래스 HW오류"
    x=x.view(B,H,W,C)
    pad_input=(H%2==1)or(W%2==1)
    if pad_input:
      x=F.pad(x,(0,0,0,W%2,0,H%2))

    x0 = x[:, 0::2, 0::2, :]
    x1 = x[:, 1::2, 0::2, :]
    x2 = x[:, 0::2, 1::2, :]
    x3 = x[:, 1::2, 1::2, :]
    x = torch.cat([x0, x1, x2, x3], -1)
    x = x.view(B, -1, 4 * C)

    x = self.norm(x)
    x = self.reduction(x)
    return x

# Multilayer perceptron (FFN)
class Mlp(nn.Module):
    def __init__(
        self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.0):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# 윈도우 멀티헤드 셀프어텐션
class WindowAttention(nn.Module):
    def __init__(
        self,
        dim,
        window_size,
        num_heads,
        qkv_bias=True,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
    ):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads)
        )

        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))
        coords_flatten = torch.flatten(coords, 1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=0.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        B_, N, C = x.shape
        qkv = (
            self.qkv(x)
            .reshape(B_, N, 3, self.num_heads, C // self.num_heads)
            .permute(2, 0, 3, 1, 4)
        )
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = q @ k.transpose(-2, -1)

        relative_position_bias = self.relative_position_bias_table[
            self.relative_position_index.view(-1)
        ].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1
        )
        relative_position_bias = relative_position_bias.permute(
            2, 0, 1
        ).contiguous()
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

# Swin Transformer 블록
class SwinTransformerBlock(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        window_size=7,
        shift_size=0,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop=0.0,
        attn_drop=0.0,
        drop_path=0.0,
        act_layer=nn.GELU,
        norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim,
            window_size=to_2tuple(self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop,
        )

        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(
            in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop
        )

        self.H = None
        self.W = None

    def forward(self, x, mask_matrix):
        B, L, C = x.shape
        H, W = self.H, self.W
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        pad_l = pad_t = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size
        x = F.pad(x, (0, 0, pad_l, pad_r, pad_t, pad_b))
        _, Hp, Wp, _ = x.shape

        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            attn_mask = mask_matrix
        else:
            shifted_x = x
            attn_mask = None

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)
        attn_windows = self.attn(x_windows, mask=attn_mask)

        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, Hp, Wp)

        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x

        if pad_r > 0 or pad_b > 0:
            x = x[:, :H, :W, :].contiguous()

        x = x.view(B, H * W, C)

        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

# Stage: BasicLayer
class BasicLayer(nn.Module):
    def __init__(
        self,
        dim,
        depth,
        num_heads,
        window_size=7,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop=0.0,
        attn_drop=0.0,
        drop_path=0.0,
        norm_layer=nn.LayerNorm,
        downsample=None,
        use_checkpoint=False,
    ):
        super().__init__()
        self.window_size = window_size
        self.shift_size = window_size // 2
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        self.blocks = nn.ModuleList(
            [
                SwinTransformerBlock(
                    dim=dim,
                    num_heads=num_heads,
                    window_size=window_size,
                    shift_size=0 if (i % 2 == 0) else window_size // 2,
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    qk_scale=qk_scale,
                    drop=drop,
                    attn_drop=attn_drop,
                    drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                    norm_layer=norm_layer,
                )
                for i in range(depth)
            ]
        )

        if downsample is not None:
            self.downsample = downsample(dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x, H, W):
        Hp = int(np.ceil(H / self.window_size)) * self.window_size
        Wp = int(np.ceil(W / self.window_size)) * self.window_size
        img_mask = torch.zeros((1, Hp, Wp, 1), device=x.device)
        h_slices = (
            slice(0, -self.window_size),
            slice(-self.window_size, -self.shift_size),
            slice(-self.shift_size, None),
        )
        w_slices = (
            slice(0, -self.window_size),
            slice(-self.window_size, -self.shift_size),
            slice(-self.shift_size, None),
        )
        cnt = 0
        for h in h_slices:
            for w in w_slices:
                img_mask[:, h, w, :] = cnt
                cnt += 1

        mask_windows = window_partition(img_mask, self.window_size)
        mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
        attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
        attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))

        for blk in self.blocks:
            blk.H, blk.W = H, W
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x, attn_mask)
            else:
                x = blk(x, attn_mask)
        if self.downsample is not None:
            x_down = self.downsample(x, H, W)
            Wh, Ww = (H + 1) // 2, (W + 1) // 2
            return x, H, W, x_down, Wh, Ww
        else:
            return x, H, W, x, H, W

# SwinTransformer
class SwinTransformer(nn.Module):
    def __init__(
        self,
        pretrain_img_size=224,
        patch_size=4,
        in_chans=3,
        embed_dim=96,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.2,
        norm_layer=nn.LayerNorm,
        ape=False,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        frozen_stages=-1,
        use_checkpoint=False,
    ):
        super().__init__()

        self.pretrain_img_size = pretrain_img_size
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm
        self.out_indices = out_indices
        self.frozen_stages = frozen_stages

        self.patch_embed = PatchEmbed(
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None,
        )

        if self.ape:
            pretrain_img_size = to_2tuple(pretrain_img_size)
            patch_size = to_2tuple(patch_size)
            patches_resolution = [
                pretrain_img_size[0] // patch_size[0],
                pretrain_img_size[1] // patch_size[1],
            ]
            self.absolute_pos_embed = nn.Parameter(
                torch.zeros(1, embed_dim, patches_resolution[0], patches_resolution[1])
            )
            trunc_normal_(self.absolute_pos_embed, std=0.02)

        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(
                dim=int(embed_dim * 2 ** i_layer),
                depth=depths[i_layer],
                num_heads=num_heads[i_layer],
                window_size=window_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[sum(depths[:i_layer]) : sum(depths[: i_layer + 1])],
                norm_layer=norm_layer,
                downsample=PatchMerging if (i_layer < self.num_layers - 1) else None,
                use_checkpoint=use_checkpoint,
            )
            self.layers.append(layer)

        num_features = [int(embed_dim * 2 ** i) for i in range(self.num_layers)]
        self.num_features = num_features

        for i_layer in out_indices:
            layer = norm_layer(num_features[i_layer])
            layer_name = f"norm{i_layer}"
            self.add_module(layer_name, layer)

        self._freeze_stages()

    def _freeze_stages(self):
        if self.frozen_stages >= 0:
            self.patch_embed.eval()
            for param in self.patch_embed.parameters():
                param.requires_grad = False
        if self.frozen_stages >= 1 and self.ape:
            self.absolute_pos_embed.requires_grad = False
        if self.frozen_stages >= 2:
            self.pos_drop.eval()
            for i in range(0, self.frozen_stages - 1):
                m = self.layers[i]
                m.eval()
                for param in m.parameters():
                    param.requires_grad = False

    def init_weights(self, pretrained=None):
        def _init_weights(m):
            if isinstance(m, nn.Linear):
                trunc_normal_(m.weight, std=0.02)
                if isinstance(m, nn.Linear) and m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LayerNorm):
                nn.init.constant_(m.bias, 0)
                nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.patch_embed(x)
        Wh, Ww = x.size(2), x.size(3)
        if self.ape:
            absolute_pos_embed = F.interpolate(
                self.absolute_pos_embed, size=(Wh, Ww), mode="bicubic"
            )
            x = (x + absolute_pos_embed).flatten(2).transpose(1, 2)
        else:
            x = x.flatten(2).transpose(1, 2)
        x = self.pos_drop(x)

        outs = {}
        for i in range(self.num_layers):
            layer = self.layers[i]
            x_out, H, W, x, Wh, Ww = layer(x, Wh, Ww)

            if i in self.out_indices:
                norm_layer = getattr(self, f"norm{i}")
                x_out = norm_layer(x_out)
                out = x_out.view(-1, H, W, self.num_features[i]).permute(0, 3, 1, 2).contiguous()
                outs["res{}".format(i + 2)] = out

        return outs

    def train(self, mode=True):
        super(SwinTransformer, self).train(mode)
        self._freeze_stages()

# 브릿지 클래스 (Backbone용)
class D2SwinTransformer(SwinTransformer):
    @configurable
    def __init__(self, *, input_shape, **kwargs):
        super().__init__(**kwargs)

        # cfg 대신 input_shape을 직접 받아 사용하도록 수정
        # self._out_features = cfg.MODEL.SWIN.OUT_FEATURES # 원본
        self._out_features = ["res2", "res3", "res4", "res5"] # 임시 하드코딩 (cfg에서 받아오도록 수정 필요)

        self._out_feature_strides = {
            "res2": 4,
            "res3": 8,
            "res4": 16,
            "res5": 32,
        }
        self._out_feature_channels = {
            "res2": self.num_features[0],
            "res3": self.num_features[1],
            "res4": self.num_features[2],
            "res5": self.num_features[3],
        }

    @classmethod
    def from_config(cls, cfg, input_shape):
        # 이 부분은 cfg 객체를 실제로 사용할 때 채워야 합니다.
        # backbone코드.ipynb의 기본값을 사용하도록 kwargs를 구성합니다.
        kwargs = {
            "pretrain_img_size": cfg.MODEL.SWIN.PRETRAIN_IMG_SIZE,
            "patch_size": cfg.MODEL.SWIN.PATCH_SIZE,
            "in_chans": input_shape.channels, # cfg.MODEL.SWIN.IN_CHANS 대신 input_shape 사용
            "embed_dim": cfg.MODEL.SWIN.EMBED_DIM,
            "depths": cfg.MODEL.SWIN.DEPTHS,
            "num_heads": cfg.MODEL.SWIN.NUM_HEADS,
            "window_size": cfg.MODEL.SWIN.WINDOW_SIZE,
            "mlp_ratio": cfg.MODEL.SWIN.MLP_RATIO,
            "qkv_bias": cfg.MODEL.SWIN.QKV_BIAS,
            "qk_scale": cfg.MODEL.SWIN.QK_SCALE,
            "drop_rate": cfg.MODEL.SWIN.DROP_RATE,
            "attn_drop_rate": cfg.MODEL.SWIN.ATTN_DROP_RATE,
            "drop_path_rate": cfg.MODEL.SWIN.DROP_PATH_RATE,
            "norm_layer": nn.LayerNorm, # cfg.MODEL.SWIN.NORM_LAYER 값에 따라 수정
            "ape": cfg.MODEL.SWIN.APE,
            "patch_norm": cfg.MODEL.SWIN.PATCH_NORM,
            "use_checkpoint": cfg.MODEL.SWIN.USE_CHECKPOINT,
            "out_indices": cfg.MODEL.SWIN.OUT_INDICES
        }
        return {"input_shape": input_shape, **kwargs}


    def forward(self, x):
        assert (
            x.dim() == 4
        ), f"입력텐서가 (N, C, H, W).가 아닌 {x.shape}"
        outputs = {}
        y = super().forward(x)
        for k in y.keys():
            if k in self._out_features:
                outputs[k] = y[k]
        return outputs

    def output_shape(self):
        return {
            name: ShapeSpec(
                channels=self._out_feature_channels[name], stride=self._out_feature_strides[name]
            )
            for name in self._out_features
        }

    @property
    def size_divisibility(self):
        return 32

In [18]:
# --- 5. 🧠 모델: 2부 (Pixel Decoder) ---
# (pixeldecoder_tem_fpn...ipynb 파일에서 복사)

# @title BasePixelDecoder (pixeldecoder_tem_fpn.ipynb)
# BasePixelDecoder는 FPN 스타일이며, TransformerEncoderPixelDecoder의 부모 클래스입니다.
class BasePixelDecoder(nn.Module):
    @configurable
    def __init__(
        self,
        input_shape: Dict[str, ShapeSpec],
        *,
        conv_dim: int,
        mask_dim: int,
        norm: Optional[Union[str, Callable]] = None,
    ):
        """
        NOTE: this interface is experimental.
        Args:
            input_shape: shapes (channels and stride) of the input features
            conv_dims: number of output channels for the intermediate conv layers.
            mask_dim: number of output channels for the final conv layer.
            norm (str or callable): normalization for all conv layers
        """
        super().__init__()

        input_shape = sorted(input_shape.items(), key=lambda x: x[1].stride)
        self.in_features = [k for k, v in input_shape]  # res2 ~ res5
        feature_channels = [v.channels for k, v in input_shape]

        lateral_convs = []
        output_convs = []

        use_bias = norm == ""
        for idx, in_channels in enumerate(feature_channels):
            if idx == len(self.in_features) - 1: # 마지막 레이어 (res5)
                output_norm = get_norm(norm, conv_dim)
                output_conv = Conv2d(
                    in_channels,
                    conv_dim,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=use_bias,
                    norm=output_norm,
                    activation=F.relu,
                )
                weight_init.c2_xavier_fill(output_conv)
                self.add_module("layer_{}".format(idx + 1), output_conv)

                lateral_convs.append(None)
                output_convs.append(output_conv)
            else: # res2, res3, res4
                lateral_norm = get_norm(norm, conv_dim)
                output_norm = get_norm(norm, conv_dim)

                lateral_conv = Conv2d(
                    in_channels, conv_dim, kernel_size=1, bias=use_bias, norm=lateral_norm
                )
                output_conv = Conv2d(
                    conv_dim,
                    conv_dim,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=use_bias,
                    norm=output_norm,
                    activation=F.relu,
                )
                weight_init.c2_xavier_fill(lateral_conv)
                weight_init.c2_xavier_fill(output_conv)
                self.add_module("adapter_{}".format(idx + 1), lateral_conv)
                self.add_module("layer_{}".format(idx + 1), output_conv)

                lateral_convs.append(lateral_conv)
                output_convs.append(output_conv)

        self.lateral_convs = lateral_convs[::-1]
        self.output_convs = output_convs[::-1]

        self.mask_dim = mask_dim
        self.mask_features = nn.Conv2d(
            conv_dim,
            mask_dim,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        weight_init.c2_xavier_fill(self.mask_features)

        self.maskformer_num_feature_levels = 3  # 3개의 스케일 사용

    @classmethod
    def from_config(cls, cfg, input_shape: Dict[str, ShapeSpec]):
        ret = {}
        # cfg가 없으면 AttributeError가 발생할 수 있으므로 try-except 추가 (뼈대용)
        try:
            ret["input_shape"] = {
                k: v for k, v in input_shape.items() if k in cfg.MODEL.SEM_SEG_HEAD.IN_FEATURES
            }
            ret["conv_dim"] = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
            ret["mask_dim"] = cfg.MODEL.SEM_SEG_HEAD.MASK_DIM
            ret["norm"] = cfg.MODEL.SEM_SEG_HEAD.NORM
        except AttributeError:
             print("경고: cfg 객체가 없거나 구조가 다릅니다. BasePixelDecoder 기본값으로 대체합니다.")
             ret["input_shape"] = {k: v for k, v in input_shape.items() if k in ["res2", "res3", "res4", "res5"]}
             ret["conv_dim"] = 256
             ret["mask_dim"] = 256
             ret["norm"] = "GN"
        return ret

    def forward_features(self, features):
        multi_scale_features = []
        num_cur_levels = 0
        y = None # Top-down 시작을 위해 초기화

        # Reverse feature maps into top-down order (from low to high resolution)
        for idx, f in enumerate(self.in_features[::-1]):
            x = features[f]
            lateral_conv = self.lateral_convs[idx]
            output_conv = self.output_convs[idx]
            if lateral_conv is None:
                y = output_conv(x)
            else:
                if y is None:
                    raise ValueError("y (top-down feature) is not initialized in BasePixelDecoder.")

                cur_fpn = lateral_conv(x)
                y = cur_fpn + F.interpolate(y, size=cur_fpn.shape[-2:], mode="nearest")
                y = output_conv(y)

            if num_cur_levels < self.maskformer_num_feature_levels:
                multi_scale_features.append(y)
                num_cur_levels += 1

        return self.mask_features(y), None, multi_scale_features

    def forward(self, features, targets=None):
        # logger = logging.getLogger(__name__)
        # logger.warning("Calling forward() may cause unpredicted behavior of PixelDecoder module.")
        return self.forward_features(features)


# @title TransformerEncoderLayer (pixeldecoder_tem_fpn.ipynb)
# Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    def __init__(
        self,
        d_model,
        nhead,
        dim_feedforward=2048,
        dropout=0.1,
        activation="relu",
        normalize_before=False,
    ):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(
        self,
        src,
        src_mask: Optional[Tensor] = None,
        src_key_padding_mask: Optional[Tensor] = None,
        pos: Optional[Tensor] = None,
    ):
        q = k = self.with_pos_embed(src, pos)
        src2 = self.self_attn(
            q, k, value=src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask
        )[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

    def forward_pre(
        self,
        src,
        src_mask: Optional[Tensor] = None,
        src_key_padding_mask: Optional[Tensor] = None,
        pos: Optional[Tensor] = None,
    ):
        src2 = self.norm1(src)
        q = k = self.with_pos_embed(src2, pos)
        src2 = self.self_attn(
            q, k, value=src2, attn_mask=src_mask, key_padding_mask=src_key_padding_mask
        )[0]
        src = src + self.dropout1(src2)
        src2 = self.norm2(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src2))))
        src = src + self.dropout2(src2)
        return src

    def forward(
        self,
        src,
        src_mask: Optional[Tensor] = None,
        src_key_padding_mask: Optional[Tensor] = None,
        pos: Optional[Tensor] = None,
    ):
        if self.normalize_before:
            return self.forward_pre(src, src_mask, src_key_padding_mask, pos)
        return self.forward_post(src, src_mask, src_key_padding_mask, pos)

# @title TransformerEncoder (pixeldecoder_tem_fpn.ipynb)
class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers, norm=None):
        super().__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(
        self,
        src,
        mask: Optional[Tensor] = None,
        src_key_padding_mask: Optional[Tensor] = None,
        pos: Optional[Tensor] = None,
    ):
        output = src

        for layer in self.layers:
            output = layer(
                output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, pos=pos
            )

        if self.norm is not None:
            output = self.norm(output)

        return output

# @title TransformerEncoderOnly (pixeldecoder_tem_fpn.ipynb)
class TransformerEncoderOnly(nn.Module):
    def __init__(
        self,
        d_model=512,
        nhead=8,
        num_encoder_layers=6,
        dim_feedforward=2048,
        dropout=0.1,
        activation="relu",
        normalize_before=False,
    ):
        super().__init__()

        encoder_layer = TransformerEncoderLayer(
            d_model, nhead, dim_feedforward, dropout, activation, normalize_before
        )
        encoder_norm = nn.LayerNorm(d_model) if normalize_before else None
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, mask, pos_embed):
        # flatten NxCxHxW to HWxNxC
        bs, c, h, w = src.shape
        src = src.flatten(2).permute(2, 0, 1)
        pos_embed = pos_embed.flatten(2).permute(2, 0, 1)
        if mask is not None:
            mask = mask.flatten(1)

        memory = self.encoder(src, src_key_padding_mask=mask, pos=pos_embed)
        return memory.permute(1, 2, 0).view(bs, c, h, w)

# @title TransformerEncoderPixelDecoder (pixeldecoder_tem_fpn.ipynb)
class TransformerEncoderPixelDecoder(BasePixelDecoder):
    @configurable
    def __init__(
        self,
        input_shape: Dict[str, ShapeSpec],
        *,
        transformer_dropout: float,
        transformer_nheads: int,
        transformer_dim_feedforward: int,
        transformer_enc_layers: int,
        transformer_pre_norm: bool,
        conv_dim: int,
        mask_dim: int,
        norm: Optional[Union[str, Callable]] = None,
    ):
        """
        NOTE: this interface is experimental.
        Args:
            input_shape: shapes (channels and stride) of the input features
            transformer_dropout: dropout probability in transformer
            transformer_nheads: number of heads in transformer
            transformer_dim_feedforward: dimension of feedforward network
            transformer_enc_layers: number of transformer encoder layers
            transformer_pre_norm: whether to use pre-layernorm or not
            conv_dims: number of output channels for the intermediate conv layers.
            mask_dim: number of output channels for the final conv layer.
            norm (str or callable): normalization for all conv layers
        """
        super().__init__(input_shape, conv_dim=conv_dim, mask_dim=mask_dim, norm=norm)

        input_shape = sorted(input_shape.items(), key=lambda x: x[1].stride)
        self.in_features = [k for k, v in input_shape]  # starting from "res2" to "res5"
        feature_strides = [v.stride for k, v in input_shape]
        feature_channels = [v.channels for k, v in input_shape]

        in_channels = feature_channels[len(self.in_features) - 1]
        self.input_proj = Conv2d(in_channels, conv_dim, kernel_size=1)
        weight_init.c2_xavier_fill(self.input_proj)
        self.transformer = TransformerEncoderOnly(
            d_model=conv_dim,
            dropout=transformer_dropout,
            nhead=transformer_nheads,
            dim_feedforward=transformer_dim_feedforward,
            num_encoder_layers=transformer_enc_layers,
            normalize_before=transformer_pre_norm,
        )
        # N_steps = conv_dim // 2 # Original incorrect line
        # --- Fix: Initialize pixel_position_embedding with conv_dim ---
        self.pe_layer = pixel_position_embedding(conv_dim)
        # --- End Fix ---

        # update layer
        use_bias = norm == ""
        output_norm = get_norm(norm, conv_dim)
        output_conv = Conv2d(
            conv_dim,
            conv_dim,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=use_bias,
            norm=output_norm,
            activation=F.relu,
        )
        weight_init.c2_xavier_fill(output_conv)
        delattr(self, "layer_{}".format(len(self.in_features)))
        self.add_module("layer_{}".format(len(self.in_features)), output_conv)
        self.output_convs[0] = output_conv

    @classmethod
    def from_config(cls, cfg, input_shape: Dict[str, ShapeSpec]):
        ret = super().from_config(cfg, input_shape)

        # (cfg가 없으면 AttributeError 발생)
        try:
            ret["transformer_dropout"] = cfg.MODEL.MASK_FORMER.DROPOUT
            ret["transformer_nheads"] = cfg.MODEL.MASK_FORMER.NHEADS
            ret["transformer_dim_feedforward"] = cfg.MODEL.MASK_FORMER.DIM_FEEDFORWARD
            ret[
                "transformer_enc_layers"
            ] = cfg.MODEL.SEM_SEG_HEAD.TRANSFORMER_ENC_LAYERS  # a separate config
            ret["transformer_pre_norm"] = cfg.MODEL.MASK_FORMER.PRE_NORM
        except AttributeError:
             print("경고: cfg 객체가 없거나 구조가 다릅니다. TransformerEncoderPixelDecoder 기본값으로 대체합니다.")
             ret["transformer_dropout"] = 0.1
             ret["transformer_nheads"] = 8
             ret["transformer_dim_feedforward"] = 1024 # 스모크 테스트 값
             ret["transformer_enc_layers"] = 3 # 스모크 테스트 값
             ret["transformer_pre_norm"] = False

        return ret

    def forward_features(self, features):
        multi_scale_features = []
        num_cur_levels = 0
        y = None # y를 None으로 초기화 (원본 코드)

        # Reverse feature maps into top-down order (from low to high resolution)
        for idx, f in enumerate(self.in_features[::-1]):
            x = features[f]
            lateral_conv = self.lateral_convs[idx]
            output_conv = self.output_convs[idx]
            if lateral_conv is None:
                transformer = self.input_proj(x)
                pos = self.pe_layer(x)
                transformer = self.transformer(transformer, None, pos)
                y = output_conv(transformer)
                # save intermediate feature as input to Transformer decoder
                transformer_encoder_features = transformer
            else:
                cur_fpn = lateral_conv(x)
                # Following FPN implementation, we use nearest upsampling here
                y = cur_fpn + F.interpolate(y, size=cur_fpn.shape[-2:], mode="nearest")
                y = output_conv(y)
            if num_cur_levels < self.maskformer_num_feature_levels:
                multi_scale_features.append(y)
                num_cur_levels += 1
        return self.mask_features(y), transformer_encoder_features, multi_scale_features

    def forward(self, features, targets=None):
        # logger = logging.getLogger(__name__)
        # logger.warning("Calling forward() may cause unpredicted behavior of PixelDecoder module.")
        return self.forward_features(features)

In [19]:
# --- 6. 🧠 모델: 3부 (Transformer Decoder) ---
# (TransformDecoder.py 파일에서 복사)

# 1단계 : masked attention + add & norm
class masked_attention(nn.Module):

    #c_dim: feature dimension = query dimension
    #n_head: 멀티해더 수
    #논문에서 dropout이 0일 때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정
    def __init__(self, C_dim, n_head, dropout=0.0):

        super(masked_attention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(C_dim, n_head, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        self.norm = nn.LayerNorm(C_dim)

        #parameter 초기화, xavier_uniform_ 사용
        self.reset_parameters()

    def reset_parameters(self):
        for par in self.parameters():
            if par.dim() > 1:
                nn.init.xavier_uniform_(par)

    # query와 key에 위치 임베딩 더하기 위해 구현
    def add_pos_embedding(self, x: Tensor, pos: Tensor) -> Tensor:
        return x if pos is None else x + pos

    # x_last: 그전 단계의 출력값
    # img_feat: 이미지 feature map(C_dim으로 차원변환 된)
    # mask: 마스크 텐서 / 키 마스크는 없는 것으로 설정 -> input으로 넣기 전에 전처리 따로 할 예정
    # query_pos: 쿼리 포지션 임베딩 (쿼리가 어떤 위치를 나타내는지 알려주는 임베딩)
    # pixel_pos: 픽셀 포지션 임베딩 (이미지 픽셀의 위치 정보를 담은 임베딩)
    def forward(self, x_last, img_feat: Tensor,
                mask: Tensor,
                query_pos: Tensor, pixel_pos: Tensor
                ):
        # masked attention
        mask_attened = self.multihead_attn(query = self.add_pos_embedding(x_last, query_pos),
                                        key = self.add_pos_embedding(img_feat, pixel_pos),
                                        value = img_feat,
                                        attn_mask = mask,
                                        key_padding_mask = None
                                        )[0]
        # add & norm
        # 논문에 나오는 add&norm 부분을 masked attention 모듈 안에 구현
        # add: x + f(x) / norm : 정규화(LayerNorm)
        x_next = x_last + self.dropout(mask_attened)
        x_next = self.norm(x_next)

        return x_next

# 2단계 : self attention + add & norm
class self_attention(nn.Module):
    #c_dim: feature dimension = query dimension
    #n_head: 멀티해더 수
    #논문에서 dropout이 0일 때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정
    def __init__(self, C_dim, n_head, dropout=0.0):
        super(self_attention, self).__init__()
        self.self_attn = nn.MultiheadAttention(C_dim, n_head, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        self.norm = nn.LayerNorm(C_dim)

        self.reset_parameters()

    #parameter 초기화, xavier_uniform_ 사용
    def reset_parameters(self):
        for par in self.parameters():
            if par.dim() > 1:
                nn.init.xavier_uniform_(par)

    # query와 key에 위치 임베딩 더하기 위해 구현
    def add_pos_embedding(self, x: Tensor, pos: Tensor) -> Tensor:
        return x if pos is None else x + pos

    # x_last: 그전 단계의 출력값
    # mask: 마스크 텐서 / 키 마스크는 없는 것으로 설정(input으로 넣기 전에 전처리 따로 할 예정)
    # query_pos: 쿼리 포지션 임베딩 (쿼리가 어떤 위치를 나타내는지 알려주는 임베딩)
    # self attention이기 때문에 key = value
    def forward(self, x_last, mask = None, query_pos =None):
        # self attention
        Query_and_Key = self.add_pos_embedding(x_last, query_pos)

        self_attened = self.self_attn(
            query = Query_and_Key,
            key = Query_and_Key,
            value = x_last,
            attn_mask = mask,
            key_padding_mask = None
            )[0]

        # add & norm
        # 논문에 나오는 add&norm 부분을 self attention 모듈 안에 구현
        # add: x + f(x) / norm : 정규화(LayerNorm)
        x_next = x_last + self.dropout(self_attened)
        x_next = self.norm(x_next)

        return x_next

# 3단계 : FFN + add & norm
class FFN(nn.Module):
    #C_dim: feature dimension
    #dim_feedforward: FFN 내부의 hidden layer 차원
    #논문에서 dropout이 0일 때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정
    def __init__(self, C_dim, dim_feedforward=2048, dropout=0.0):
        super(FFN, self).__init__()

        self.first_linear = nn.Linear(C_dim, dim_feedforward)
        self.second_linear = nn.Linear(dim_feedforward, C_dim)

        self.dropout = nn.Dropout(dropout)

        self.activation = F.relu

        self.norm = nn.LayerNorm(C_dim)
        #parameter 초기화, xavier_uniform_ 사용
        self.reset_parameters()

    def reset_parameters(self):
        for par in self.parameters():
            if par.dim() > 1:
                nn.init.xavier_uniform_(par)

    def forward(self, x_last):
        # FFN
        # FFN = relu(w1x + b1)W2 + b2
        ffn = self.second_linear(self.dropout(self.activation(self.first_linear(x_last))))

        # add & norm
        # 논문에 나오는 add&norm 부분을 FFN 모듈 안에 구현
        # add: x + f(x) / norm : 정규화(LayerNorm)
        x_next = x_last + self.dropout(ffn)
        x_next = self.norm(x_next)

        return x_next

# 마스킹 위해 Multi-Layer Perceptron(MLP) 구현
class mlp(nn.Module):
    def __init__(self, n_layers, input_dim, hidden_dim, output_dim):
        super(mlp, self).__init__()

        self.n_layers = n_layers
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.layers = self.make_layers()
        self.last_layer = nn.Linear(hidden_dim, output_dim)

    def make_layers(self):
        mlp_layers = [nn.Linear(self.input_dim, self.hidden_dim)]

        for _ in range(self.n_layers-2):
            mlp_layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))

        return nn.ModuleList(mlp_layers)

    def forward(self, input):
        for layer in self.layers:
            input = F.relu(layer(input))

        output = self.last_layer(input)
        return output

# pixel position embedding을 하기 위한 모델 구현
# sinusoidal position embedding 사용
class pixel_position_embedding(nn.Module):
    def __init__(self, d_model):
        super(pixel_position_embedding, self).__init__()
        self.d_model = d_model

    #feature_map : (B, C, H, W)
    def forward(self, feature_map: Tensor):
        #sinusoidal을 사용하고 [H | W] 같이 H와 W를 나누어서 입력해야 해서 d_model이 4의 배수여야 함
        assert self.d_model % 4 == 0, "차원이 4의 배수가 아닙니다."

        #좌표 격자 만들기
        B, H, W = feature_map.size(0), feature_map.size(2), feature_map.size(3)
        H_coord, W_coord = torch.meshgrid(torch.arange(1, H+1, dtype=torch.float32, device=feature_map.device),
                                          torch.arange(1, W+1, dtype=torch.float32, device=feature_map.device),
                                          indexing='ij')

        #좌표 스케일링 0~2ㅠ
        H_coord = H_coord.unsqueeze(0).expand(B, -1, -1) / (H + 1e-6) * (2 * math.pi)
        W_coord = W_coord.unsqueeze(0).expand(B, -1, -1) / (W + 1e-6) * (2 * math.pi)

        #pos 나눌 분모 만들기
        half_d = self.d_model // 2
        denom = torch.arange(0, half_d, dtype=torch.float32, device=feature_map.device) // 2
        denom = 10000 ** (2*denom / half_d)

        #pos/10000^(2i/d_model)
        pos_H = H_coord.unsqueeze(-1) / denom
        pos_W = W_coord.unsqueeze(-1) / denom

        #sin, cos 적용(짝수는 sin, 홀수는 cos)
        pos_H[:, :, :, 0::2].sin_()
        pos_H[:, :, :, 1::2].cos_()
        pos_W[:, :, :, 0::2].sin_()
        pos_W[:, :, :, 1::2].cos_()

        #H와 W 합치기 및 (B, d_model, H, W)로 차원 변경
        pos_embedding = torch.cat([pos_H, pos_W], dim=-1).permute(0, 3, 1, 2)

        # output shape : (B, d_model, H, W)
        return pos_embedding

class Transformer_Decoder(nn.Module):
    """
    파라미터 설명란
    C_dim: feature dimension = query dimension
    n_head: 멀티헤더 어텐션의 헤더 수
    dim_feedforward: FFN 내부의 hidden layer 차원
    channel_dim_list: 각 feature map위 채널 차원수 리스트 [1/32, 1/16, 1/8, 1/4]
    mask_dim : 1/4 해상도의 C 차원
    L: 디코더 반복 횟수 따라서 총 3L개의 레이어가 있다.(논문에 3을 사용하여 default를 3으로 설정)
    dropout: 드롭아웃 (논문에서 0일때가 퍼포먼스가 가장 좋았다고 해서 default를 0으로 설정)
    n_query: 쿼리의 수 (논문에서 100을 사용하여 default를 100으로 설정)
    n_class: 구분해야 하는 클래수 수(건물 하나의 클래스만 할 것이므로 default 1로 설정)
    """
    def __init__(self,
                C_dim: int,
                n_head: int,
                dim_feedforward: int,
                channel_dim_list: list,
                mask_dim: int,
                L: int = 3,
                dropout: float = 0.0,
                n_query: int = 100,
                n_class: int = 1):
        super(Transformer_Decoder, self).__init__()

        #기본 파라미터 저장
        self.channel_dim_list = channel_dim_list
        self.L = L
        self.C_dim = C_dim
        self.n_head = n_head
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.mask_dim = mask_dim
        self.n_class = n_class

        # 디코더 레이어 만들기, 3개 층을 L번 반복
        self.masked_attn_layers = nn.ModuleList()
        self.self_attn_layers = nn.ModuleList()
        self.ffn_layers = nn.ModuleList()
        self.make_layers()

        #쿼리 임베딩 만들기(학습가능 임베딩으로 만들기)
        self.n_query = n_query
        self.query_feat_embed = nn.Embedding(n_query, C_dim)
        self.query_pos_embed = nn.Embedding(n_query, C_dim)

        #해상도 임베딩 만들기(3가지 해상도에 대해 학습가능 임베딩으로 만들기)
        self.resolution_embed = nn.Embedding(3, C_dim)

        #각 feature map의 채널과 C_dim이 다를 때 차원을 변환하는 모듈 만들기
        self.projection_modules = self.make_projection_modules()

        #픽셀 위치 임베딩 모듈 만들기
        self.pixel_pos_embedding = pixel_position_embedding(d_model=C_dim)


        self.Cdim_norm = nn.LayerNorm(C_dim)

        #class + 배경 백터
        self.class_predic = nn.Linear(C_dim, self.n_class+1)
        #mask embed: maske_dim(1/4 feature map의 C)로 바꿈
        self.mask_embed = mlp(n_layers=3, input_dim=self.C_dim,
                              hidden_dim=self.C_dim, output_dim=self.mask_dim)



    def make_layers(self):
        for _ in range(self.L * 3):
            self.masked_attn_layers.append(masked_attention(
                C_dim = self.C_dim,
                n_head = self.n_head,
                dropout = self.dropout
                ))
            self.self_attn_layers.append(self_attention(
                C_dim = self.C_dim,
                n_head = self.n_head,
                dropout = self.dropout
                ))
            self.ffn_layers.append(FFN(
                C_dim = self.C_dim,
                dim_feedforward = self.dim_feedforward,
                dropout = self.dropout
                ))

    def make_projection_modules(self):
        output = nn.ModuleList()
        for channel_dim in self.channel_dim_list:
            if channel_dim != self.C_dim:
                output.append(nn.Conv2d(channel_dim, self.C_dim, kernel_size=1))
                weight_init.c2_xavier_fill(output[-1])
            else:
                output.append(nn.Identity())
        return output

    def predic_class_mask_masking(self, x, feat_map4: Tensor, masking_size):
        x_normed = self.Cdim_norm(x)
        class_predic = self.class_predic(x_normed)
        mask_embed = self.mask_embed(x_normed)

        # 1/4 해상도 feature map과 mask embed 내적
        # (B, Q, C) @ (B, C, H/4, W/4) -> (B, Q, H/4, W/4)
        mask_predic = torch.einsum("bqc, bchw -> bqhw", mask_embed, feat_map4)

        #(B, Q, H_i, W_i) -> (B, Q, H_i*W_i)
        masking = F.interpolate(mask_predic, size=masking_size, mode='bilinear', align_corners= False).sigmoid().flatten(2)
        masking = masking.unsqueeze(0).repeat(self.n_head, 1, 1, 1).flatten(0,1)
        #0.5이상이면 F 미만이면 T
        #T이면 마스킹을 해라, F면 attention을 하라
        masking = (masking < 0.5).bool()
        #masking이 mask_predic으로 부터 파생되는 값이므로 detach 실행
        masking = masking.detach()

        return class_predic, mask_predic, masking

    #feature_pyramid: 픽셀 디코더에서 만든 [1/32, 1/16, 1/8, 1/4] 해상도의 feature map 리스트
    #각 feature map은 (B, C, H, W) 형태
    def forward(self, feature_pyramid: list[Tensor]):
        #feature map의 채널 차원 확인
        assert [feature_pyramid[i].size(1) for i in range(len(feature_pyramid))] == self.channel_dim_list, "feature map의 채널 차원이 맞지 않습니다."

        #쿼리에 맞게 차원 변환된 feature map + 해상도 embed
        proj_feat_list = []
        #pixel에 따른 embed 리스트(상수)
        pos_emb_list = []
        #해상도 크기 리스트 (H, W)
        HW_list = []
        #배치사이즈
        B = feature_pyramid[0].size(0)

        for i in range(3):
            #(B, C, H, W) -> (B, C_dim, HxW) + (1, C_dim, 1) -> (B, C_dim, HxW)
            proj_feat_list.append(
                self.projection_modules[i](feature_pyramid[i]).flatten(2)
                + self.resolution_embed.weight[i][None, :, None])
            #(B, C_dim, HxW)->(B, HxW, C_dim)
            proj_feat_list[-1] = proj_feat_list[-1].permute(0, 2, 1)

            #(B, C_dim, H, W) -> (B, C_dim, HxW)->(B, HxW, C_dim)
            pos_emb_list.append(self.pixel_pos_embedding(feature_pyramid[i]).flatten(2).permute(0, 2, 1))
            #HW의 int값을 가지는 리스트
            HW_list.append(feature_pyramid[i].shape[-2:])

        # (Q, C_dim)-> (B, Q, C_dim)
        # layer에 입력할 쿼리에 layer에서 더할 쿼리 위치 embed 설정
        x = self.query_feat_embed.weight.unsqueeze(0).repeat(B, 1, 1)
        query_pos_embed = self.query_pos_embed.weight.unsqueeze(0).repeat(B, 1, 1)

        class_predic_list = []
        mask_predic_list = []

        class_predic, mask_predic, masking = self.predic_class_mask_masking(x, feat_map4=feature_pyramid[-1], masking_size=HW_list[0])

        class_predic_list.append(class_predic)
        mask_predic_list.append(mask_predic)

        #총 3L 레이어 : (1/32->1/16->1/8)*3
        for i in range(self.L):
            for j in range(3):
                masking[torch.where(torch.all(masking, dim=-1))] = False

                index = 3*i + j
                #mask attention layer
                x = self.masked_attn_layers[index](
                    x_last=x,
                    img_feat=proj_feat_list[j],
                    mask = masking,
                    query_pos = query_pos_embed,
                    pixel_pos = pos_emb_list[j]
                    )
                #self attention layer
                x = self.self_attn_layers[index](
                    x_last=x,
                    query_pos=query_pos_embed,
                    mask = None
                )
                #FFN layer
                x = self.ffn_layers[index](
                    x_last=x
                )

                # 다음 단계 마스킹 예상 + 마스크, 클래스 예상
                class_predic, mask_predic, masking = self.predic_class_mask_masking(x, feat_map4=feature_pyramid[-1], masking_size=HW_list[(j+1)%3])

                #결과 리스트에 추가
                class_predic_list.append(class_predic)
                mask_predic_list.append(mask_predic)

        assert len(class_predic_list) == 3*self.L + 1, "예측 클래스 개수 안 맞음"
        assert len(mask_predic_list) == 3*self.L + 1, "예측 마스크 개수 안 맞음"

        final_out = {
                'predicted_class' : class_predic_list[-1],
                'predicted_mask' : mask_predic_list[-1],
                'auxiliary_out' : [
                                   {"predicted_class":c, "predicted_mask":m}
                                   for c, m in zip(class_predic_list[:-1], mask_predic_list[:-1])
                                  ]

                    }
        return final_out

In [20]:
# --- 7. ⚖️ 모델: 4부 (Matcher 및 Criterion) ---

# --- from matcher.py ---
import torch
# (nn, Tensor, F는 이미 상단에서 import 했습니다)
# from torch import nn, Tensor
# import torch.nn.functional as F

# !pip install scipy (필요시 설치)
from scipy.optimize import linear_sum_assignment

# @title (Helpers from matcher.py)
# matcher.py에 정의된 JIT 스크립트 함수들을 클래스 외부에 정의합니다.
# (참고: @torch.jit.script 데코레이터는 클래스 메서드 내에서 직접 사용하기보다
#  별도 함수로 정의된 것을 호출할 때 가장 잘 작동합니다.)

@torch.jit.script
def query_mask_sigmoid_ce_loss(mask_prob: Tensor, target_mask: Tensor, K: int):
            assert mask_prob.shape[-1]==K and target_mask.shape[-1]==K, "K가 안 맞습니다."

            #정답이 1일 때의 loss 계산 -> (Q, K)
            calc_one_loss = F.binary_cross_entropy_with_logits(
                mask_prob, torch.ones_like(mask_prob), reduction='none'
            )
            #정답이 0일 때의 loss 계산 -> (Q, K)
            calc_zero_loss = F.binary_cross_entropy_with_logits(
                mask_prob, torch.zeros_like(mask_prob), reduction='none'
            )

            loss_sum = (
                        #모두 1일때의 loss 값과 실제 마스크를 곱해 실제 1일 때의 loss 구함
                        #(Q, K) @ (K, N) -> (Q,  N)
                        torch.einsum('qk,nk->qn', calc_one_loss, target_mask)
                        #모두 0일때의 loss 값과 실제 마스크를 곱해(0과 1을 바꾼) 실제 0일때의 loss 구함
                        #(Q, K) @ (K, N) -> (Q,  N)
                        + torch.einsum('qk,nk->qn', calc_zero_loss, (1 - target_mask))
                        )
            return loss_sum / K

@torch.jit.script
def query_dice_loss(mask_prob: Tensor, target_mask: Tensor, K: int):
            assert mask_prob.shape[-1]==K and target_mask.shape[-1]==K, "K가 안 맞습니다."
            assert mask_prob.ndim == target_mask.ndim == 2 , "차원이 안 맞습니다."

            mask_prob = mask_prob.sigmoid()
            #2(A^B)
            numerator = 2*(torch.einsum('qk,nk->qn', mask_prob, target_mask))
            #|A|+|B|
            denominator = mask_prob.sum(-1).unsqueeze(-1) + target_mask.sum(-1).unsqueeze(0)
            # 1- 2(A^B)/(|A|+|B|) + zero division error 방지
            loss = 1 -(numerator+1)/(denominator+1)
            return loss

# @title hungarian_matcher (matcher.py)
class hungarian_matcher(nn.Module):
    #각각의 loss의 가중치들을 파라미터로 가짐
    def __init__(self, class_cost_weight:float = 1, mask_cost_weight:float = 1, dice_cost_weight:float = 1):
        super().__init__()

        self.weight_c = class_cost_weight
        self.weight_m = mask_cost_weight
        self.weight_d = dice_cost_weight
        assert not (self.weight_c == 0 and self.weight_d == 0 and self.weight_m == 0), "가중치가 모두 0입니다."

        # JIT 함수를 클래스 속성으로 할당
        self.mask_sigmoid_ce_loss_jit = query_mask_sigmoid_ce_loss
        self.dice_loss_jit = query_dice_loss

    """
    Trans_dec_oupts: Transform Decoder에서 만든 최종 output
    {
    'predicted_class' : (B, Q, 2)
    'predicted_mask' : (B, Q, H/4, W/4),
    'auxiliary_out' : [...]
    }

    targets: 이미지의 라벨 [{instance_class: tensor, mask: tensor[N,H,W]}, {...} , {...}, ...]
    K: 램덤으로 확인할 좌표(픽셀) 수 -> 논문에서 112*112를 사용해서 default 값으로 사용
    """
    def forward(self, trans_dec_outputs: dict, targets: list[dict] , K: int = 112*112):

        B, Q = trans_dec_outputs['predicted_class'].shape[0:2]
        assert B == trans_dec_outputs['predicted_mask'].shape[0] and Q == trans_dec_outputs['predicted_mask'].shape[1], "B랑 Q가 이상합니다."

        B_matching_cost = []
        for i_b in range(B):
            #(Q, 2)
            class_prob = trans_dec_outputs['predicted_class'][i_b].softmax(-1)
            #(N)
            target_class = targets[i_b]['instance_class'].to(class_prob.device)
            #(Q, N=1)
            class_cost = -class_prob[:, target_class]

            #(Q, H/4, H/4) -> (Q, 1, H/4, W/4)
            mask_prob = trans_dec_outputs['predicted_mask'][i_b].unsqueeze(1)
            #(N, H, W) -> (N, 1, H, W)
            target_mask = targets[i_b]['mask'].to(mask_prob).unsqueeze(1)

            # [-1, 1) 사이 램덤 좌표 K개 생성 (1, 1, K, 2)
            rand_coords = torch.rand(1, 1, K, 2, device=class_prob.device)*2 -1

            mask_prob = F.grid_sample(
                mask_prob,
                rand_coords.repeat(mask_prob.shape[0], 1, 1, 1),
                align_corners = False
            )
            #(Q, 1, 1, K) -> (Q,K)
            mask_prob = mask_prob.squeeze(1).squeeze(1)

            target_mask = F.grid_sample(
                target_mask,
                rand_coords.repeat(target_mask.shape[0], 1, 1, 1),
                align_corners = False
            )
            #(N, 1, 1, K) -> (N, K)
            target_mask = target_mask.squeeze(1).squeeze(1)

            #float 32로 연산
            with torch.cuda.amp.autocast(enabled=False):
                mask_prob = mask_prob.float()
                target_mask = target_mask.float()
                # mask match 비용 계산
                mask_cost = self.mask_sigmoid_ce_loss_jit(mask_prob, target_mask, K)
                # dice match 비용 계산
                dice_cost = self.dice_loss_jit(mask_prob, target_mask, K)

            weighted_cost = self.weight_m*mask_cost + self.weight_d*dice_cost + self.weight_c*class_cost
            weighted_cost = weighted_cost.cpu()
            B_matching_cost.append(linear_sum_assignment(weighted_cost.detach().cpu().numpy()))

        #[(쿼리 인덱스 탠서, 타겟 인덱스 탠서), ...] -> batch 개수만큼 투플 있는 리스트
        return [(torch.as_tensor(p, dtype=torch.int64), torch.as_tensor(t, dtype=torch.int64))
                    for p, t in B_matching_cost
        ]

# --- from criterion.py ---

# @title (Helpers from criterion.py)
# criterion.py에 정의된 JIT 스크립트 함수들을 클래스 외부에 정의합니다.
@torch.jit.script
def mask_sigmoid_ce_loss(mask_prob: Tensor, mask_target: Tensor):
    assert mask_prob.size(-1) == mask_target.size(-1), "예측과 타겟의 사이즈가 안 맞습니다."

    loss = F.binary_cross_entropy_with_logits(mask_prob, mask_target, reduction='none')

    return loss.mean(-1).sum() / mask_prob.size(0)

@torch.jit.script
def dice_loss(mask_prob: Tensor, mask_target: Tensor):
    mask_prob = mask_prob.sigmoid()

    numerator = 2 * (mask_prob * mask_target).sum(-1)
    denominator = mask_prob.sum(-1) + mask_target.sum(-1)

    loss = 1-(numerator+1) / (denominator+1)

    return loss.sum() / mask_prob.size(0)

# @title criterion (criterion.py)
class criterion(nn.Module):
    def __init__(self, oversample_ratio: float ,important_sample_ratio: float, class_cost_weight: float=1, mask_cost_weight: float=1, dice_cost_weight: float=1, eos_weigt: float=1,
                K: int=112*112 ):
        super().__init__()

        self.matcher = hungarian_matcher()

        self.important_sample_ratio = important_sample_ratio
        self.oversample_ratio = oversample_ratio

        self.w_m = mask_cost_weight
        self.w_c = class_cost_weight
        self.w_d = dice_cost_weight

        self.w_e = eos_weigt

        background_ratio = torch.ones(2)
        background_ratio[-1] = self.w_e
        self.background_ratio = background_ratio

        self.K = K

        self.mask_loss_jit = mask_sigmoid_ce_loss
        self.dice_loss_jit = dice_loss

    #중요도 샘플링을 하기 위한 점수를 계산하는 함수
    #(N, K)
   # (7번 셀의 criterion 클래스 내부를 이걸로 수정하세요)
# (class criterion(nn.Module): ... self.dice_loss_jit = dice_loss ... 이후)

    #중요도 샘플링을 하기 위한 점수를 계산하는 함수
    #(N, K)
    def calc_uncertainty_score(self, coords):
        #애매한 픽셀(확률 0.5에 가까운)을 높은 점수를 주기 위해 -절댓값
        score = -(torch.abs(coords))
        return score

    # [수정됨] get_query_idx1을 호출할 때 device를 넘겨줍니다.
    def calc_class_loss(self, matched: list[tuple], predicted_class: Tensor):
        B, Q = predicted_class.size(0), predicted_class.size(1)

        # [수정] predicted_class.device를 인자로 전달
        idx = self.get_query_idx1(matched, Q, device=predicted_class.device)

        target_class = torch.ones(B*Q, dtype=torch.int64, device=predicted_class.device)
        target_class[idx] = 0

        # (수정) background_ratio를 predicted_class와 동일한 device로 이동
        class_loss = F.cross_entropy(predicted_class.reshape(B*Q, 2), target_class, self.background_ratio.to(predicted_class.device))
        output = {'class_loss': class_loss}
        return output

    # [수정됨] device 인자를 받고, CPU 텐서를 .to(device)로 옮깁니다.
    def get_query_idx1(self, matched: list[tuple], Q:int, device): # 'device' 인자 추가
        # (수정) matched 리스트가 비어있을 경우(매칭된 것이 없을 때) 빈 텐서 반환
        if not matched or all(len(q) == 0 for q, _ in matched):
            # [수정] 'cpu' 대신 전달받은 'device' 사용
            return torch.tensor([], dtype=torch.int64, device=device)

        # [수정] matcher가 CPU로 반환한 'q'를 .to(device)로 GPU에 올림
        idx = torch.cat([i*Q + q.to(device) for i, (q, _) in enumerate(matched)])
        return idx

    # [수정됨] device 인자를 받고, CPU 텐서를 .to(device)로 옮깁니다.
    def get_query_idx2(self, matched: list[tuple], device): # 'device' 인자 추가
        # (수정) matched 리스트가 비어있을 경우 빈 텐서 반환
        if not matched or all(len(q) == 0 for q, _ in matched):
            # [수정] 'cpu' 대신 전달받은 'device' 사용
            empty_tensor = torch.tensor([], dtype=torch.int64, device=device)
            return empty_tensor, empty_tensor

        # [수정] matcher가 CPU로 반환한 'q'를 .to(device)로 GPU에 올림
        batch_idx = torch.cat([torch.full_like(q.to(device), i) for i, (q, _) in enumerate(matched)])
        query_idx = torch.cat([q.to(device) for (q, _) in matched])
        return batch_idx, query_idx

    # [수정됨] get_query_idx2와 get_target_idx를 호출할 때 device를 넘겨줍니다.
    def calc_mask_dice_loss(self, matched: list[tuple], predicted_mask:Tensor, label: list[dict]):

        # [수정] predicted_mask.device를 인자로 전달
        q_idx = self.get_query_idx2(matched, device=predicted_mask.device)

        # (수정) 매칭된 것이 없으면 0 반환
        if q_idx[0].numel() == 0:
            return {
                'mask_loss': torch.tensor(0.0, device=predicted_mask.device),
                'dice_loss': torch.tensor(0.0, device=predicted_mask.device)
            }

        #(B, Q, H/4, W/4) -> (T, 1, H/4, W/4) -> T: 확인할 총 마스크 수
        predicted_mask = predicted_mask[q_idx].unsqueeze(1)

        # [수정] predicted_mask.device를 인자로 전달
        t_idx = self.get_target_idx(matched, device=predicted_mask.device)

        #(T, H, W)- -> (T, 1, H, W)
        target_mask = torch.cat([t['mask'] for t in label], dim=0)
        #순서대로 바꾸기
        target_mask = torch.index_select(target_mask, dim=0, index=t_idx).unsqueeze(1) # <- 여기가 문제의 지점

        with torch.no_grad():
            coords = self.get_coords(predicted_mask).unsqueeze(1)
            target_point = F.grid_sample(
                target_mask.float(), # (수정) grid_sample은 float 입력 필요
                coords,
                align_corners=False
            ).squeeze(1).squeeze(1)

        predict_point = F.grid_sample(
            predicted_mask,
            coords,
            align_corners=False
        ).squeeze(1).squeeze(1)

        loss = {
                'mask_loss': self.mask_loss_jit(predict_point, target_point),
                'dice_loss': self.dice_loss_jit(predict_point, target_point)
            }

        return loss

    # [수정됨] device 인자를 받고, CPU 텐서를 .to(device)로 옮깁니다.
    def get_target_idx(self, matched: list[tuple], device): # 'device' 인자 추가
        target = [t for (_, t) in matched]

        # (수정) 매칭된 것이 없으면 빈 텐서 반환
        if not target or all(t.numel() == 0 for t in target):
             # [수정] 'cpu' 대신 전달받은 'device' 사용
             return torch.tensor([], dtype=torch.int64, device=device)

        size = [t.numel() for t in target]

        # (수정) device 일치
        # device = target[0].device  <- 이 부분이 CPU를 참조하게 되어 버그 발생. 삭제.

        # [수정] matcher가 CPU로 반환한 't'를 .to(device)로 GPU에 올림
        idx = torch.cat([t.to(device) for t in target])

        # [수정] offset 계산도 전달받은 'device' 사용
        offset = torch.tensor([0] + list(torch.cumsum(torch.tensor(size[:-1]), 0)), device=device)
        offset = torch.repeat_interleave(offset, torch.tensor(size, device=device))

        return idx + offset

    # (get_coords 함수와 forward 함수는 수정할 필요 없이 그대로 두면 됩니다)
    # ... def get_coords(self, ...): ...
    # ... def forward(self, ...): ...

    def get_coords(self, predicted_mask):
        assert self.oversample_ratio >= 1, "oversample_ratio는 1보다 커야 합니다."
        assert 0 <= self.important_sample_ratio <= 1, "important_sample_ratio는 0과 1 사이이어야 합니다."

        n_sample = int(self.oversample_ratio * self.K)
        n_important = int(self.important_sample_ratio * self.K)
        n_random = self.K - n_important

        T = predicted_mask.shape[0]
        #(T, n_sample, 2)
        sample_coord = torch.rand(T, n_sample, 2, device=predicted_mask.device)*2 - 1
         #(T, 1, n_sample, 2)
        sample_coord_grid = sample_coord.unsqueeze(1)
        #(T, 1, 1, n_sample)
        sample_point = F.grid_sample(
            predicted_mask,
            sample_coord_grid,
            align_corners=False
        )
        #(T, n_sample)
        sample_point = sample_point.squeeze(1).squeeze(1)

        score = self.calc_uncertainty_score(sample_point)
        idx = torch.topk(score, k=n_important, dim=1)[1]

        offset = torch.arange(T, dtype=torch.long, device=predicted_mask.device)*n_sample
        idx += offset.unsqueeze(-1)

        important_coord = sample_coord.view(-1, 2)[idx.view(-1), :].view(T, n_important, 2)

        if n_random > 0:
            random_coord = torch.rand(T, n_random, 2, device=predicted_mask.device)*2 -1
            final_coord = torch.cat([important_coord, random_coord], dim=1)
        else:
            final_coord = important_coord

        return final_coord

    def forward(self, trans_dec_outputs: dict, targets: list[dict] ):

        auxiliary_out = trans_dec_outputs['auxiliary_out']

        #matcher: [(쿼리 인덱스 탠서, 타겟 인덱스 탠서), ...] -> batch 개수만큼 투플 있는 리스트
        matched = self.matcher(trans_dec_outputs, targets, K=self.K)

        final_loss = {}
        final_loss.update(self.calc_class_loss(matched, trans_dec_outputs['predicted_class']))
        final_loss.update(self.calc_mask_dice_loss(matched, trans_dec_outputs['predicted_mask'], targets))

        for i, aux in enumerate(auxiliary_out):
            # (수정) aux loss 계산 시에도 K값 전달
            aux_matched = self.matcher(aux, targets, K=self.K)
            temp = self.calc_class_loss(aux_matched, aux['predicted_class'])
            temp.update(self.calc_mask_dice_loss(aux_matched, aux['predicted_mask'], targets))
            temp = {k + f"_{i}": v for k, v in temp.items()}
            final_loss.update(temp)

        return final_loss

In [21]:
# --- 7. 🚀 모델: 4부 (Mask2Former 조립) ---
# (누락된 부분이므로, 표준 아키텍처에 맞춰 새로 작성)

class Mask2Former(nn.Module):
    """
    Backbone (Swin) + Pixel Decoder (TEncoderPixelDecoder) + Transformer Decoder (Transformer_Decoder)
    를 조립하는 최종 Mask2Former 모델입니다.
    """

    # 이 클래스는 from_config를 사용하지 않고,
    # 필요한 설정값(cfg)을 직접 받아 하위 모듈을 초기화합니다.
    def __init__(self, backbone_cfg, pixel_decoder_cfg, transformer_decoder_cfg, input_shape):
        super().__init__()

        # --- 1. Backbone (Swin) 초기화 ---
        # input_shape은 (C, H, W)를 가정 (e.g., ShapeSpec(channels=3))
        # D2SwinTransformer.from_config는 cfg와 input_shape을 받음
        self.backbone = D2SwinTransformer(
            **D2SwinTransformer.from_config(backbone_cfg, input_shape)
        )

        # 백본의 출력 shape을 가져옴 (e.g., {"res2": ShapeSpec, ...})
        backbone_output_shape = self.backbone.output_shape()

        # --- 2. Pixel Decoder (FPN + Encoder) 초기화 ---
        # TransformerEncoderPixelDecoder.from_config는 cfg와 backbone_output_shape을 받음
        pixel_decoder_params = TransformerEncoderPixelDecoder.from_config(
            pixel_decoder_cfg, backbone_output_shape
        )
        self.pixel_decoder = TransformerEncoderPixelDecoder(**pixel_decoder_params)

        # --- 3. Transformer Decoder 초기화 ---
        # Transformer_Decoder는 cfg가 아닌 개별 인자를 받습니다.
        # (주의: channel_dim_list와 C_dim, mask_dim이 Pixel Decoder와 일치해야 함)

        # Pixel Decoder의 FPN 출력 채널 (conv_dim)
        # (res5 -> 1/32, res4 -> 1/16, res3 -> 1/8)
        # (주의: TransformerEncoderPixelDecoder의 multi_scale_features는 3개만 반환함)
        conv_dim = pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM

        # Pixel Decoder의 최종 마스크 특징 채널 (mask_dim)
        # (res2 -> 1/4)
        mask_dim = pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.MASK_DIM

        # Transformer_Decoder에 전달할 채널 리스트
        # [1/32, 1/16, 1/8, 1/4] 스케일에 해당
        # (주의: Transformer_Decoder의 projection_modules[i]는 feature_pyramid[i]에 적용됨)
        # (주의: Transformer_Decoder의 forward는 3개의 feature map(0,1,2)과 1개의 mask map(3)을 사용함)
        # (주의: TransformerEncoderPixelDecoder의 multi_scale_features는 3개(conv_dim)를,
        #  mask_features는 1개(mask_dim)를 반환함. channel_dim_list와 순서/개수가 맞지 않음)

        # --- [충돌 해결] ---
        # 제공된 Transformer_Decoder의 forward는 4개의 feature_pyramid 입력을 받아
        # [0], [1], [2] (3개)는 cross-attention에, [3] (1개)는 마스크 예측에 사용합니다.

        # TransformerEncoderPixelDecoder의 multi_scale_features (3개)를 [0, 1, 2]에,
        # mask_features (1개)를 [3]에 매핑합니다.

        # (Pixel Decoder의 multi_scale_features는 [res5(1/32), res4(1/16), res3(1/8)] 순서)
        # (Pixel Decoder의 mask_features는 [res2(1/4)] 스케일)

        # channel_dim_list (Transformer_Decoder가 기대하는 입력 채널)
        # [1/32, 1/16, 1/8, 1/4]
        # (res5, res4, res3의 채널은 pixel_decoder의 conv_dim)
        # (res2의 채널은 pixel_decoder의 mask_dim)
        decoder_channel_dim_list = [conv_dim, conv_dim, conv_dim, mask_dim]

        # Transformer_Decoder의 C_dim (내부 Hidden Dim)
        # pixel_decoder_cfg의 conv_dim과 동일하게 맞추는 것이 일반적입니다.
        decoder_C_dim = pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM

        # Transformer_Decoder의 mask_dim (MLP 출력 Dim)
        # pixel_decoder_cfg의 mask_dim과 동일해야 합니다.
        decoder_mask_dim = pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.MASK_DIM

        self.transformer_decoder = Transformer_Decoder(
            C_dim=decoder_C_dim,
            n_head=transformer_decoder_cfg.MODEL.MASK_FORMER.NHEADS,
            dim_feedforward=transformer_decoder_cfg.MODEL.MASK_FORMER.DIM_FEEDFORWARD,
            channel_dim_list=decoder_channel_dim_list,
            mask_dim=decoder_mask_dim,
            L=transformer_decoder_cfg.MODEL.MASK_FORMER.DEC_LAYERS, # (cfg에 이 항목이 필요합니다)
            dropout=transformer_decoder_cfg.MODEL.MASK_FORMER.DROPOUT,
            n_query=transformer_decoder_cfg.MODEL.MASK_FORMER.NUM_OBJECT_QUERIES, # (cfg에 이 항목이 필요합니다)
            n_class=transformer_decoder_cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES # (cfg에 이 항목이 필요합니다)
        )

    def forward(self, x):
        # 1. 백본 실행 (Swin)
        # (Input: [B, 3, H, W])
        # (Output: {"res2": [B,C2,H/4,W/4], "res3": [B,C3,H/8,W/8], ...})
        features = self.backbone(x)

        # 2. 픽셀 디코더 실행 (TransformerEncoderPixelDecoder)
        # (Input: features dict)
        # (Output: mask_features [B, mask_dim, H/4, W/4],
        #          transformer_encoder_features [B, conv_dim, H/32, W/32],
        #          multi_scale_features [List of 3 Tensors: 1/32, 1/16, 1/8 scale])
        mask_features, _, multi_scale_features = self.pixel_decoder.forward_features(features)

        # 3. 트랜스포머 디코더 실행 (Transformer_Decoder)
        # Transformer_Decoder는 [1/32, 1/16, 1/8, 1/4] 스케일의 리스트를 기대합니다.

        # (Pixel Decoder의 multi_scale_features는 [res5(1/32), res4(1/16), res3(1/8)])
        # (Pixel Decoder의 mask_features는 res2(1/4) 스케일)

        # (주의) TransformerEncoderPixelDecoder의 multi_scale_features는
        # [res5, res4, res3] (High-res to Low-res가 아님)
        # 원본 BasePixelDecoder의 순서는 low-to-high ([res5, res4, res3]) 입니다.
        # Transformer_Decoder의 forward는 [1/32, 1/16, 1/8] 순서를 기대합니다.

        # feature_pyramid 리스트 생성: [feat_1/32, feat_1/16, feat_1/8, feat_1/4]
        feature_pyramid = [
            multi_scale_features[0], # 1/32 (res5)
            multi_scale_features[1], # 1/16 (res4)
            multi_scale_features[2], # 1/8 (res3)
            mask_features            # 1/4 (res2, mask_dim 채널)
        ]

        # (Input: [List of 4 Tensors])
        # (Output: {'predicted_class': ..., 'predicted_mask': ..., 'auxiliary_out': ...})
        outputs = self.transformer_decoder(feature_pyramid)

        return outputs

In [22]:
# --- 8. 🚀 실행 및 스모크 테스트 ---
#
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob
import os
import types

# (셀 1, 3, 4, 5, 6, 7에서 정의된 모든 클래스가 필요합니다)
# (e.g., ShapeSpec, D2SwinTransformer, TransformerEncoderPixelDecoder, Transformer_Decoder, criterion)

# --- 1. 학습용 Dataset 및 DataLoader 준비 ---

class BuildingTrainingDataset(Dataset):
    """
    전처리로 생성된 개별 .pt 파일들을 읽어오는 학습용 데이터셋
    """
    def __init__(self, data_dir):
        self.image_dir = os.path.join(data_dir, "images")
        self.label_dir = os.path.join(data_dir, "labels_dict")
        self.file_list = sorted(glob.glob(os.path.join(self.label_dir, "*.pt")))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        label_path = self.file_list[idx]
        file_name = os.path.basename(label_path)
        image_path = os.path.join(self.image_dir, file_name)

        # 이미지 로드 (float 변환 및 정규화)
        # (주의) 원본 .pt가 uint8이었다면 float() 변환 및 255.0 나누기 필요
        # (2번 셀의 create_final_dataset_per_file에서 .float()로 저장했으므로 /255.0만 필요)
        image = torch.load(image_path) / 255.0

        # 라벨 딕셔너리 로드
        target = torch.load(label_path)

        # (중요) 라벨 키 변경 (criterion 코드와 맞춤)
        # (2번 셀에서 'masks'로 저장했으므로 이름 변경 불필요)
        # target['masks'] = target.pop('mask')
        # (2번 셀에서 'instance_class'로 저장했으므로 이름 변경 필요)
        target['instance_class'] = target.pop('instance_class')
        # (수정) criterion.py의 calc_mask_dice_loss가 'mask' 키를 참조합니다.
        # (수정) 2번 셀의 convert_to_target_format이 'masks'로 저장합니다.
        # (수정) 7번 셀의 matcher.py가 'mask' 키를 참조합니다. -> 7번 셀 수정 필요
        # (임시) 2번 셀의 'masks'를 7번 셀에 맞게 'mask'로 변경
        target['mask'] = target.pop('masks')


        return image, target

def collate_fn(batch):
    """
    배치 내의 이미지와 타겟(딕셔너리 리스트)을 묶어줍니다.
    """
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets

# --- 2. 설정값 (Configs) 준비 (9번 셀에서 복사) ---

# 2.1: 백본(Swin) 설정
backbone_cfg = _CfgNode()
backbone_cfg.MODEL = _CfgNode()
backbone_cfg.MODEL.SWIN = _CfgNode()
backbone_cfg.MODEL.SWIN.PRETRAIN_IMG_SIZE = 224
backbone_cfg.MODEL.SWIN.PATCH_SIZE = 4
backbone_cfg.MODEL.SWIN.EMBED_DIM = 96
backbone_cfg.MODEL.SWIN.DEPTHS = [2, 2, 6, 2]
backbone_cfg.MODEL.SWIN.NUM_HEADS = [3, 6, 12, 24]
backbone_cfg.MODEL.SWIN.WINDOW_SIZE = 7
backbone_cfg.MODEL.SWIN.MLP_RATIO = 4.0
backbone_cfg.MODEL.SWIN.QKV_BIAS = True
backbone_cfg.MODEL.SWIN.QK_SCALE = None
backbone_cfg.MODEL.SWIN.DROP_RATE = 0.0
backbone_cfg.MODEL.SWIN.ATTN_DROP_RATE = 0.0
backbone_cfg.MODEL.SWIN.DROP_PATH_RATE = 0.3
backbone_cfg.MODEL.SWIN.APE = False
backbone_cfg.MODEL.SWIN.PATCH_NORM = True
backbone_cfg.MODEL.SWIN.USE_CHECKPOINT = False
backbone_cfg.MODEL.SWIN.OUT_INDICES = (0, 1, 2, 3)

# 2.2: 픽셀 디코더 (TEncoderPixelDecoder) 설정
pixel_decoder_cfg = _CfgNode()
pixel_decoder_cfg.MODEL = _CfgNode()
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD = _CfgNode()
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.IN_FEATURES = ["res2", "res3", "res4", "res5"]
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM = 256
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.MASK_DIM = 256
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.NORM = "GN"
pixel_decoder_cfg.MODEL.MASK_FORMER = _CfgNode()
pixel_decoder_cfg.MODEL.MASK_FORMER.DROPOUT = 0.1
pixel_decoder_cfg.MODEL.MASK_FORMER.NHEADS = 8
pixel_decoder_cfg.MODEL.MASK_FORMER.DIM_FEEDFORWARD = 1024
pixel_decoder_cfg.MODEL.SEM_SEG_HEAD.TRANSFORMER_ENC_LAYERS = 3
pixel_decoder_cfg.MODEL.MASK_FORMER.PRE_NORM = False

# 2.3: 트랜스포머 디코더 (Transformer_Decoder) 설정
transformer_decoder_cfg = _CfgNode()
transformer_decoder_cfg.MODEL = _CfgNode()
transformer_decoder_cfg.MODEL.MASK_FORMER = _CfgNode()
transformer_decoder_cfg.MODEL.MASK_FORMER.NHEADS = 8
transformer_decoder_cfg.MODEL.MASK_FORMER.DIM_FEEDFORWARD = 2048
transformer_decoder_cfg.MODEL.MASK_FORMER.DEC_LAYERS = 3
transformer_decoder_cfg.MODEL.MASK_FORMER.DROPOUT = 0.0
transformer_decoder_cfg.MODEL.MASK_FORMER.NUM_OBJECT_QUERIES = 100
transformer_decoder_cfg.MODEL.SEM_SEG_HEAD = _CfgNode()
transformer_decoder_cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1

# --- 3. 핵심 구성요소 초기화 ---

print("--- 🧠 학습 준비 시작 ---")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"DEVICE: {DEVICE}")

# [중요]
# 스모크 테스트(256)와 달리, 전처리된 타일(1024) 크기로 모델 입력 shape 설정
INPUT_H, INPUT_W = 1024, 1024
input_shape = ShapeSpec(channels=3, height=INPUT_H, width=INPUT_W)

# 3.1: 모델 (Mask2Former)
model = Mask2Former(
    backbone_cfg,
    pixel_decoder_cfg,
    transformer_decoder_cfg,
    input_shape
).to(DEVICE)

# 3.2: 손실 함수 (Criterion) - 7번 셀에서 정의됨
# (7번 셀의 기본값을 사용)
model_criterion = criterion(
    oversample_ratio=3.0,
    important_sample_ratio=0.75,
    class_cost_weight=1.0,
    mask_cost_weight=1.0,
    dice_cost_weight=1.0,
    eos_weigt=0.1 # '배경' 클래스 가중치 (원본 논문 값)
).to(DEVICE) # criterion도 device로 보내는 것이 좋습니다.

# 3.3: 옵티마이저 (Optimizer)
# Transformer 모델은 AdamW가 표준입니다.
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,          # 학습률 (Learning Rate)
    weight_decay=1e-4 # 가중치 감쇠 (Weight Decay)
)

# 3.4: 데이터 로더 (DataLoader)
TRAIN_DATA_DIR = "dummy_dataset_output" # 2번 셀에서 만든 더미데이터 폴더
BATCH_SIZE = 1 # [중요] 1024x1024 이미지는 크므로, Colab 메모리를 위해 배치 1로 설정

train_dataset = BuildingTrainingDataset(TRAIN_DATA_DIR)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # 학습 시에는 데이터를 섞어줍니다.
    collate_fn=collate_fn
)

print("✅ 모델, 손실함수, 옵티마이저, 데이터로더 초기화 완료!")

# --- 4. 실제 학습(Training) 루프 ---

# (테스트용) 더미데이터가 작으므로 10 에포크(Epoch)만 돌려봅니다.
NUM_EPOCHS = 10

print(f"--- 🚀 학습 시작 (총 {NUM_EPOCHS} 에포크) ---")

model.train() # 모델을 "학습 모드"로 설정 (Dropout 등 활성화)

for epoch in range(NUM_EPOCHS):

    # train_loader가 데이터를 1개씩 (batch_size=1) 꺼내옵니다.
    # images: [1, 3, 1024, 1024]
    # targets: [{'mask': [N, 1024, 1024], 'instance_class': [N]}] (길이 1짜리 리스트)
    for i, (images, targets) in enumerate(train_loader):

        # 1. [필수] 데이터와 타겟을 GPU(DEVICE)로 이동
        images = images.to(DEVICE)

        targets_gpu = []
        for t in targets:
            # (주의) 7번 셀의 criterion이 'mask'와 'instance_class'를 기대합니다.
            targets_gpu.append({
                'mask': t['mask'].to(DEVICE),
                'instance_class': t['instance_class'].to(DEVICE)
            })

        # 2. [필수] 옵티마이저 그래디언트 초기화
        optimizer.zero_grad()

        # 3. [핵심] 모델 순전파 (Forward Pass)
        # 이미지를 모델에 넣어 예측값(outputs)을 받습니다.
        outputs = model(images)

        # 4. [핵심] 손실 계산 (Loss Calculation)
        # 예측(outputs)과 정답(targets_gpu)을 비교하여 손실을 계산합니다.
        # (7번 셀의 criterion.forward가 호출됨)
        loss_dict = model_criterion(outputs, targets_gpu)

        # 5. [핵심] 총 손실(Total Loss) 계산
        # criterion은 여러 종류의 손실(class, mask, dice)을 딕셔너리로 반환
        # 이들을 모두 더해서 최종 손실값(스칼라)으로 만듭니다.
        total_loss = sum(loss_dict.values())

        # 6. [핵심] 역전파 (Backward Pass)
        # "모델이 어디서 틀렸는지" 계산합니다.
        total_loss.backward()

        # 7. [핵심] 모델 업데이트 (Optimizer Step)
        # 계산된 그래디언트를 바탕으로 모델의 가중치를 갱신합니다.
        optimizer.step()

        # 8. (로깅) 현재 상태 출력
        if i % 1 == 0: # (더미데이터는 배치가 1개라 매번 출력됨)
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{i+1}/{len(train_loader)}], Loss: {total_loss.item():.4f}")

    print(f"--- Epoch {epoch+1} 완료 ---")

print("🎉 학습 루프가 성공적으로 종료되었습니다!")

--- 🧠 학습 준비 시작 ---
DEVICE: cuda
✅ 모델, 손실함수, 옵티마이저, 데이터로더 초기화 완료!
--- 🚀 학습 시작 (총 10 에포크) ---


/tmp/ipython-input-1559942682.py:119: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


Epoch [1/10], Batch [1/1], Loss: 24.3005
--- Epoch 1 완료 ---
Epoch [2/10], Batch [1/1], Loss: 16.1963
--- Epoch 2 완료 ---
Epoch [3/10], Batch [1/1], Loss: 15.7089
--- Epoch 3 완료 ---
Epoch [4/10], Batch [1/1], Loss: 15.7469
--- Epoch 4 완료 ---
Epoch [5/10], Batch [1/1], Loss: 15.7604
--- Epoch 5 완료 ---
Epoch [6/10], Batch [1/1], Loss: 15.4849
--- Epoch 6 완료 ---
Epoch [7/10], Batch [1/1], Loss: 15.2452
--- Epoch 7 완료 ---
Epoch [8/10], Batch [1/1], Loss: 14.9601
--- Epoch 8 완료 ---
Epoch [9/10], Batch [1/1], Loss: 14.7057
--- Epoch 9 완료 ---
Epoch [10/10], Batch [1/1], Loss: 14.4920
--- Epoch 10 완료 ---
🎉 학습 루프가 성공적으로 종료되었습니다!


In [23]:
# --- 9. 후처리 ---
#

@torch.no_grad() # 추론 시에는 그래디언트 계산이 필요 없습니다.
def post_process_predictions(outputs, target_size_hw, class_threshold=0.7, mask_threshold=0.5):
    """
    모델의 원시 출력(logits)을 받아, 필터링, 업샘플링, 이진화를 거쳐
    최종 고해상도 이진 마스크 리스트를 반환합니다.

    Args:
        outputs (dict): model(x)의 반환값.
                        {'predicted_class': [B, Q, C+1], 'predicted_mask': [B, Q, H, W]}
        target_size_hw (tuple): (height, width) e.g., (1024, 1024)
        class_threshold (float): '건물' 클래스(logit 0번)의 확률 임계값
        mask_threshold (float): 마스크를 0 또는 1로 결정하는 임계값

    Returns:
        list[dict]: 배치 내 각 이미지에 대한 결과 리스트.
                    각 딕셔너리는 {'scores': [N], 'labels': [N], 'masks': [N, H_target, W_target]}
                    (N: 임계값을 통과한 객체 수)
    """

    # 1. 모델 출력 분리
    # (B, Q, 2) -> (B, Q, 2)
    pred_logits = outputs['predicted_class']
    # (B, Q, 64, 64) -> (B, Q, 64, 64)
    pred_masks = outputs['predicted_mask']

    B, Q, _ = pred_logits.shape
    H_small, W_small = pred_masks.shape[-2:]

    # 2. 클래스 확률 계산 및 필터링
    # Softmax를 적용해 확률로 변환 (B, Q, 2)
    probs = F.softmax(pred_logits, dim=-1)

    # '건물' 클래스(인덱스 0)의 확률 (B, Q)
    # (참고: criterion 코드에서 정답 클래스를 0으로 설정했음)
    scores = probs[..., 0]

    # 결과를 저장할 리스트
    results = []

    # 3. 배치 내 각 이미지에 대해 후처리 수행
    for i in range(B):
        image_scores = scores[i]       # (Q,)
        image_masks_low_res = pred_masks[i] # (Q, 64, 64)

        # 3.1. 필터링: '건물' 확률이 임계값(class_threshold)보다 높은 쿼리만 선택
        keep = image_scores > class_threshold

        filtered_scores = image_scores[keep]          # (N,)
        filtered_masks_low_res = image_masks_low_res[keep] # (N, 64, 64)

        # 만약 임계값을 넘는 건물이 하나도 없다면
        if filtered_masks_low_res.shape[0] == 0:
            results.append({
                'scores': torch.tensor([]),
                'labels': torch.tensor([]),
                'masks': torch.empty(0, target_size_hw[0], target_size_hw[1])
            })
            continue

        # 3.2. 업샘플링 (저해상도 -> 고해상도)
        # F.interpolate는 [N, C, H, W] 형태를 기대하므로, 채널(C=1) 차원을 추가
        # (N, 64, 64) -> (N, 1, 64, 64)
        upsampled_masks = F.interpolate(
            filtered_masks_low_res.unsqueeze(1),
            size=target_size_hw,      # (1024, 1024)
            mode="bilinear",        # 부드럽게 보간
            align_corners=False
        )
        # (N, 1, 1024, 1024) -> (N, 1024, 1024)
        upsampled_masks = upsampled_masks.squeeze(1)

        # 3.3. 이진화 (Binarization)
        # (중요) 모델이 출력한 마스크(pred_masks)는 logit이므로,
        # sigmoid를 적용해 0~1 사이 확률로 만들고, mask_threshold(0.5)로 이진화
        final_binary_masks = (upsampled_masks.sigmoid() > mask_threshold).to(torch.uint8)

        # 3.4. 레이블 생성 (모두 '건물' 클래스 0)
        filtered_labels = torch.zeros_like(filtered_scores, dtype=torch.long)

        results.append({
            'scores': filtered_scores,
            'labels': filtered_labels,
            'masks': final_binary_masks
        })

    return results

    # (9번 셀의 스모크 테스트가 실행되었다고 가정)
# model, dummy_input, outputs 가 이미 존재함

print("\n--- 🚀 후처리 테스트 시작 ---")

# 1. 후처리 함수로 고해상도 이진 마스크 생성
TARGET_H, TARGET_W = 1024, 1024 # 원본 타일 크기
processed_results = post_process_predictions(outputs, (TARGET_H, TARGET_W), class_threshold=0.7)

# 'processed_results'는 리스트입니다. (배치 크기=1이므로 0번 인덱스 사용)
result_item = processed_results[0]
binary_masks_tensor = result_item['masks'] # (N, 1024, 1024)
scores = result_item['scores']           # (N,)

print(f"✅ 마스크 복원 성공!")
print(f"   -> 100개 쿼리 중 {binary_masks_tensor.shape[0]}개의 건물을 찾음 (Threshold=0.7)")
print(f"   -> 최종 마스크 텐서 형태: {binary_masks_tensor.shape}")


# 2. 다각형 변환 테스트
if binary_masks_tensor.shape[0] > 0:
    # (중요) 실제로는 Dataset에서 'tile_transform'을 불러와야 합니다.
    # 여기서는 테스트를 위해 픽셀 좌표(1,1)를 사용하는 가짜(dummy) transform을 만듭니다.
    dummy_geo_transform = rasterio.transform.from_origin(
        west=0, north=0, xsize=1, ysize=1
    )

    polygons = convert_masks_to_polygons(binary_masks_tensor, dummy_geo_transform)

    print(f"\n✅ 다각형 변환 성공!")
    print(f"   -> {len(polygons)}개의 다각형(Shapely 객체) 생성됨")
    if polygons:
        print("   -> (예시) 0번 다각형의 WKT(Well-Known Text) 표현:")
        print(f"      {polygons[0].wkt[:100]}...") # WKT: POLYGON ((...))
else:
    print("\nℹ️ 찾은 건물이 없어 다각형 변환을 건너뜁니다.")


--- 🚀 후처리 테스트 시작 ---
✅ 마스크 복원 성공!
   -> 100개 쿼리 중 0개의 건물을 찾음 (Threshold=0.7)
   -> 최종 마스크 텐서 형태: torch.Size([0, 1024, 1024])

ℹ️ 찾은 건물이 없어 다각형 변환을 건너뜁니다.
